In [ ]:
# Setup Completo - Instalação e Imports
# Execute esta célula primeiro sempre que reiniciar o kernel

# 1. Instalação de Dependências

import subprocess
import sys

def install_package(package_name, import_name=None):
    
    if import_name is None:
        import_name = package_name
    
    try:
        __import__(import_name)
        print(f"    {package_name} já instalado")
        return True
    except ImportError:
        print(f"   Instalando {package_name}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name, "-q"])
            print(f"   {package_name} instalado com sucesso")
            return True
        except Exception as e:
            print(f"   Erro ao instalar {package_name}: {e}")
            return False


# Lista de pacotes necessários
packages_to_install = [
    ("ultralytics", "ultralytics"),  # YOLO
    ("opencv-python", "cv2"),        # OpenCV
    ("seaborn", "seaborn"),         # Visualização
    ("scikit-learn", "sklearn"),     # ML metrics
    ("pillow", "PIL"),              # Processamento de imagens
    ("pyyaml", "yaml"),             # YAML parsing
]

failed_installs = []
for package, import_name in packages_to_install:
    if not install_package(package, import_name):
        failed_installs.append(package)

# 2. Imports Principais

# Bibliotecas do sistema
import os
import sys
import time
import warnings
import shutil
import random

# Manipulação de dados e caminhos
import json
import yaml
from pathlib import Path
from collections import defaultdict, Counter

# Computação científica
import numpy as np
import pandas as pd

# Visualização
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

# Processamento de imagens
import cv2
from PIL import Image

# Machine Learning
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_curve, 
    f1_score, 
    confusion_matrix, 
    classification_report, 
    roc_curve, 
    auc
)  

# YOLO (principal)
from ultralytics import YOLO

# 3. Configuração do Ambiente

# Suprimir warnings desnecessários
warnings.filterwarnings('ignore')

# Configurar matplotlib
plt.style.use('default')  # Mais compatível
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

# Configurar seaborn
sns.set_palette("husl")

# Verificar GPU

# PyTorch CUDA
if torch.cuda.is_available():
    print(f"    GPU: {torch.cuda.get_device_name(0)}")
    print(f"    VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("   GPU: Não disponível (usando CPU)")

# Verificar versões importantes

version_info = [
    ("Python", sys.version.split()[0]),
    ("NumPy", np.__version__),
    ("Pandas", pd.__version__),
    ("PyTorch", torch.__version__),
    ("OpenCV", cv2.__version__),
]

import ultralytics
version_info.append(("Ultralytics", ultralytics.__version__))

for lib, version in version_info:
    print(f"   • {lib}: {version}")

# 4. Configurações Específicas

# Configurações de reprodutibilidade
def set_seeds(seed=42):
   
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # Para determinismo no YOLO (quando possível)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seeds(42)

# Classes do projeto
CLASS_NAMES = ['COM_corante', 'SEM_corante']
CLASS_COLORS = {'COM_corante': 'blue', 'SEM_corante': 'red'}

# Paths comuns
COMMON_PATHS = {
    'dataset': '/kaggle/working/yolo_dataset',
    'models': '/kaggle/working/yolo_training', 
    'optimization': '/kaggle/working/yolo_optimization',
    'output': '/kaggle/working'
}

In [ ]:
# Bloco 1: Setup e Análise Exploratória dos Dados
# Pipeline: YOLO (Detecção e Classificação)

# Configuração de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


print(" Bloco 1: Setup e Análise Exploratória")
print(" Pipeline: YOLO (Detecção e Classificação)")


# 1. Configuração de Paths e Constantes

# IMPORTANTE: Ajustar estes paths conforme dataset
BASE_PATH = "/kaggle/input/dataset2-tcc/DatasetV2"

PATHS = {
    'train': {
        'json': f"{BASE_PATH}/train/_annotations.coco.json",
        'images': f"{BASE_PATH}/train/"
    },
    'valid': {
        'json': f"{BASE_PATH}/valid/_annotations.coco.json", 
        'images': f"{BASE_PATH}/valid/"
    },
    'test': {
        'json': f"{BASE_PATH}/test/_annotations.coco.json",
        'images': f"{BASE_PATH}/test/"
    }
}

# Mapeamento de Classes: 4 originais → 2 binárias
CLASS_MAPPING = {
    'blue cell': 'COM_corante',
    'grumo azul': 'COM_corante', 
    'normal cell': 'SEM_corante',
    'grumo normal': 'SEM_corante',
    'corante': None,  # ignorar
    'cells': None     # ignorar
}

# Classes finais para classificação
BINARY_CLASSES = ['COM_corante', 'SEM_corante']

print(f" Dataset path configurado: {BASE_PATH}")
print(f" Mapeamento binário: {len(BINARY_CLASSES)} classes")
print(f"   • COM_corante: blue cell + grumo azul")
print(f"   • SEM_corante: normal cell + grumo normal")

# 2. FUNÇÕES AUXILIARES

def load_coco_annotations(json_path):
   
    
    with open(json_path, 'r') as f:
        coco_data = json.load(f)
    
    # Extrair informações
    images = {img['id']: img for img in coco_data['images']}
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
    annotations = coco_data['annotations']
    
    print(f" Carregado: {len(images)} imagens, {len(annotations)} anotações")
    
    return coco_data, images, categories, annotations        
    

def extract_annotation_stats(annotations, categories, class_mapping):
    
    stats = {
        'original_classes': defaultdict(int),
        'binary_classes': defaultdict(int),
        'bbox_sizes': [],
        'ignored_classes': defaultdict(int)
    }
    
    valid_annotations = []
    
    for ann in annotations:
        # Classe original
        class_id = ann['category_id']
        class_name = categories[class_id]
        stats['original_classes'][class_name] += 1
        
        # Mapeamento binário
        binary_class = class_mapping.get(class_name)
        
        if binary_class is not None:
            stats['binary_classes'][binary_class] += 1
            
            # Estatísticas da bounding box
            bbox = ann['bbox']  # [x, y, width, height]
            area = bbox[2] * bbox[3]
            stats['bbox_sizes'].append({
                'width': bbox[2],
                'height': bbox[3], 
                'area': area,
                'class': binary_class
            })
            
            valid_annotations.append({
                'annotation_id': ann['id'],
                'image_id': ann['image_id'],
                'bbox': bbox,
                'area': area,
                'original_class': class_name,
                'binary_class': binary_class
            })
        else:
            stats['ignored_classes'][class_name] += 1
    
    return stats, valid_annotations

def visualize_class_distribution(stats, split_name):
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # 1. Classes originais
    orig_classes = list(stats['original_classes'].keys())
    orig_counts = list(stats['original_classes'].values())
    
    axes[0].bar(orig_classes, orig_counts, alpha=0.7)
    axes[0].set_title(f'{split_name} - Classes Originais')
    axes[0].set_ylabel('Número de Anotações')
    axes[0].tick_params(axis='x', rotation=45)
    
    # 2. Classes binárias
    binary_classes = list(stats['binary_classes'].keys())
    binary_counts = list(stats['binary_classes'].values())
    
    colors = ['skyblue', 'lightcoral']
    axes[1].bar(binary_classes, binary_counts, color=colors, alpha=0.7)
    axes[1].set_title(f'{split_name} - Classes Binárias (Final)')
    axes[1].set_ylabel('Número de Anotações')
    
    # Adicionar percentuais
    total = sum(binary_counts)
    for i, (cls, count) in enumerate(zip(binary_classes, binary_counts)):
        pct = (count/total)*100
        axes[1].text(i, count + max(binary_counts)*0.01, 
                    f'{count}\n({pct:.1f}%)', 
                    ha='center', va='bottom', fontweight='bold')
    
    # 3. Classes ignoradas (se houver)
    if stats['ignored_classes']:
        ign_classes = list(stats['ignored_classes'].keys())
        ign_counts = list(stats['ignored_classes'].values())
        
        axes[2].bar(ign_classes, ign_counts, color='gray', alpha=0.7)
        axes[2].set_title(f'{split_name} - Classes Ignoradas')
        axes[2].set_ylabel('Número de Anotações')
        axes[2].tick_params(axis='x', rotation=45)
    else:
        axes[2].text(0.5, 0.5, 'Nenhuma classe\nfoi ignorada', 
                    ha='center', va='center', transform=axes[2].transAxes,
                    fontsize=12, alpha=0.6)
        axes[2].set_title(f'{split_name} - Classes Ignoradas')
    
    plt.tight_layout()
    plt.show()

def analyze_bbox_sizes(bbox_data):
    
    df = pd.DataFrame(bbox_data)
    
    print(f"\n Análise de Bounding Boxes:")    
    
    # Estatísticas gerais
    print(f"Total de bounding boxes válidas: {len(df)}")
    print(f"\nEstatísticas de tamanho:")
    print(f"Width  - Min: {df['width'].min():.1f}, Max: {df['width'].max():.1f}, Média: {df['width'].mean():.1f}")
    print(f"Height - Min: {df['height'].min():.1f}, Max: {df['height'].max():.1f}, Média: {df['height'].mean():.1f}")
    print(f"Area   - Min: {df['area'].min():.1f}, Max: {df['area'].max():.1f}, Média: {df['area'].mean():.1f}")
    
    # Por classe
    print(f"\nPor classe:")
    for cls in df['class'].unique():
        subset = df[df['class'] == cls]
        print(f"  {cls}:")
        print(f"    Width média: {subset['width'].mean():.1f} ± {subset['width'].std():.1f}")
        print(f"    Height média: {subset['height'].mean():.1f} ± {subset['height'].std():.1f}")
        print(f"    Area média: {subset['area'].mean():.1f} ± {subset['area'].std():.1f}")
    
    # Visualização
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Distribuição de larguras
    for cls in df['class'].unique():
        subset = df[df['class'] == cls]
        axes[0,0].hist(subset['width'], alpha=0.6, label=cls, bins=20)
    axes[0,0].set_title('Distribuição de Larguras')
    axes[0,0].set_xlabel('Width (pixels)')
    axes[0,0].legend()
    
    # Distribuição de alturas
    for cls in df['class'].unique():
        subset = df[df['class'] == cls]
        axes[0,1].hist(subset['height'], alpha=0.6, label=cls, bins=20)
    axes[0,1].set_title('Distribuição de Alturas')
    axes[0,1].set_xlabel('Height (pixels)')
    axes[0,1].legend()
    
    # Scatter width vs height
    colors = {'COM_corante': 'blue', 'SEM_corante': 'red'}
    for cls in df['class'].unique():
        subset = df[df['class'] == cls]
        axes[1,0].scatter(subset['width'], subset['height'], 
                         alpha=0.6, label=cls, color=colors.get(cls, 'gray'))
    axes[1,0].set_title('Width vs Height')
    axes[1,0].set_xlabel('Width (pixels)')
    axes[1,0].set_ylabel('Height (pixels)')
    axes[1,0].legend()
    
    # Distribuição de áreas (log scale)
    for cls in df['class'].unique():
        subset = df[df['class'] == cls]
        axes[1,1].hist(np.log10(subset['area']), alpha=0.6, label=cls, bins=20)
    axes[1,1].set_title('Distribuição de Áreas (log10)')
    axes[1,1].set_xlabel('log10(Area)')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()
    
    return df

# 3. Carregamento e Análise dos Dados

print(f"\n Carregando Dados dos 3 Splits...")

all_stats = {}
all_annotations = {}

for split_name, paths in PATHS.items():
    print(f"\n Processando split: {split_name.upper()}")    
    
    # Carregar anotações COCO
    coco_data, images, categories, annotations = load_coco_annotations(paths['json'])    
        
    # Extrair estatísticas
    stats, valid_annotations = extract_annotation_stats(annotations, categories, CLASS_MAPPING)
    
    # Armazenar
    all_stats[split_name] = stats
    all_annotations[split_name] = valid_annotations
    
    print(f" Resumo do {split_name}:")
    print(f"   • Total de imagens: {len(images)}")
    print(f"   • Total de anotações: {len(annotations)}")
    print(f"   • Anotações válidas: {len(valid_annotations)}")
    
    # Classes binárias
    for cls, count in stats['binary_classes'].items():
        total_binary = sum(stats['binary_classes'].values())
        pct = (count/total_binary)*100
        print(f"   • {cls}: {count} ({pct:.1f}%)")
    
    # Classes ignoradas
    if stats['ignored_classes']:
        print(f"   • Classes ignoradas: {dict(stats['ignored_classes'])}")

# 4. Análise Consolidada

print(f"\n Análise dos Dados")

# Consolidar estatísticas
total_stats = {
    'images': 0,
    'annotations': 0,
    'valid_annotations': 0,
    'binary_classes': defaultdict(int)
}

for split_name, stats in all_stats.items():
    total_stats['valid_annotations'] += len(all_annotations[split_name])
    for cls, count in stats['binary_classes'].items():
        total_stats['binary_classes'][cls] += count

print(f" Dataset Completo:")
print(f"   • Total de anotações válidas: {total_stats['valid_annotations']}")

total_binary = sum(total_stats['binary_classes'].values())
for cls, count in total_stats['binary_classes'].items():
    pct = (count/total_binary)*100
    print(f"   • {cls}: {count} ({pct:.1f}%)")

# Verificar balanceamento
com_pct = (total_stats['binary_classes']['COM_corante']/total_binary)*100
sem_pct = (total_stats['binary_classes']['SEM_corante']/total_binary)*100
balance_ratio = min(com_pct, sem_pct) / max(com_pct, sem_pct)

print(f"\n  Análise de Balanceamento:")
print(f"   • COM_corante: {com_pct:.1f}%")
print(f"   • SEM_corante: {sem_pct:.1f}%")
print(f"   • Ratio de balanceamento: {balance_ratio:.3f}")

print(f"\n Bloco 1 Carregado")

In [ ]:
# Bloco 2 : Preparação dos Dados
# Pipeline: YOLO (Detecção e Classificação)

print(" Bloco 2: Preparação dos Dados")
print(" Objetivos: Análise + Visualização + Conversão + Extração")

# 1. Configurações e Parâmetros

# Parâmetros para extração de patches
PATCH_SIZE = 224  # Padrão para transfer learning
OUTPUT_BASE = "/kaggle/working"  # Diretório de saída no Kaggle

# Mapeamento de classes para YOLO (0, 1)
CLASS_MAPPING_REVERSE = {
    'COM_corante': 0,
    'SEM_corante': 1
}

CLASS_NAMES = ['COM_corante', 'SEM_corante']

print(f"\n Configuração Inicial:")
print(f"   • Tamanho do patch: {PATCH_SIZE}x{PATCH_SIZE}")
print(f"   • Classes YOLO: {CLASS_NAMES}")
print(f"   • Diretório de saída: {OUTPUT_BASE}")

# 2. Funções Utilitárias

def load_image_safely(image_path):    
    
    if os.path.exists(image_path):
        image = cv2.imread(image_path)
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            return image
    print(f" Imagem não encontrada: {image_path}")
    return None

def create_directories():
    
    print(f"\n Criando Estrutura de Diretórios")
    
    directories = {
        'yolo_dataset': Path(OUTPUT_BASE) / "yolo_dataset",
        'patches_dataset': Path(OUTPUT_BASE) / "patches_dataset"
    }
    
    # Estrutura YOLO
    yolo_base = directories['yolo_dataset']
    for split in ['train', 'valid', 'test']:
        (yolo_base / "images" / split).mkdir(parents=True, exist_ok=True)
        (yolo_base / "labels" / split).mkdir(parents=True, exist_ok=True)
    
    # Estrutura Patches
    patches_base = directories['patches_dataset']
    for split in ['train', 'valid', 'test']:
        for class_name in CLASS_NAMES:
            (patches_base / split / class_name).mkdir(parents=True, exist_ok=True)
    
    for name, path in directories.items():
        print(f"    {name}: {path}")
    
    return directories

def create_yolo_dataset_yaml(output_dir, class_names):
    
    yaml_content = f"""# Dataset YOLO - TCC Células Microscópicas
# Gerado automaticamente - Bloco 2

path: {output_dir}
train: images/train
val: images/valid  
test: images/test

# Classes
nc: {len(class_names)}
names: {class_names}

# Informações adicionais
description: "Classificação de células microscópicas com/sem corante azul de metileno"
version: "1.0"
date: "2025"
"""
    
    yaml_path = Path(output_dir) / "dataset.yaml"
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)
    
    return yaml_path

# 3. Visualização e Análise

def visualize_annotations_sample(split_name, images_dict, annotations_list, images_path, n_samples=6):
    
    print(f"\n Visualizando Amostras - {split_name.upper()}")    
    
    # Selecionar imagens aleatórias
    image_ids = list(images_dict.keys())
    if len(image_ids) < n_samples:
        n_samples = len(image_ids)
    
    random.seed(42)  # Reprodutibilidade
    sample_ids = random.sample(image_ids, n_samples)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    colors = {'COM_corante': 'blue', 'SEM_corante': 'red'}
    
    for idx, image_id in enumerate(sample_ids):
        if idx >= len(axes):
            break
            
        # Carregar imagem
        image_info = images_dict[image_id]
        image_path = os.path.join(images_path, image_info['file_name'])
        image = load_image_safely(image_path)     
        
        # Encontrar anotações desta imagem
        image_annotations = [ann for ann in annotations_list if ann['image_id'] == image_id]
        
        # Plotar imagem
        axes[idx].imshow(image)
        
        # Plotar bounding boxes
        class_counts = defaultdict(int)
        for ann in image_annotations:
            bbox = ann['bbox']  # [x, y, width, height]
            binary_class = ann['binary_class']
            class_counts[binary_class] += 1
            
            # Criar retângulo
            rect = patches.Rectangle(
                (bbox[0], bbox[1]), bbox[2], bbox[3],
                linewidth=2, edgecolor=colors[binary_class], 
                facecolor='none', alpha=0.8
            )
            axes[idx].add_patch(rect)
            
            # Adicionar label
            axes[idx].text(bbox[0], bbox[1]-5, binary_class, 
                          color=colors[binary_class], fontsize=8, 
                          fontweight='bold', alpha=0.9)
        
        # Título com estatísticas
        title = f'{image_info["file_name"]}\n'
        title += f'COM: {class_counts["COM_corante"]}, SEM: {class_counts["SEM_corante"]}'
        axes[idx].set_title(title, fontsize=10)
        axes[idx].axis('off')
    
    # Ocultar axes vazios
    for idx in range(len(sample_ids), len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle(f'Amostras do Split {split_name.upper()} - Bounding Boxes Anotadas', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return sample_ids

def analyze_bbox_detailed(all_annotations):
    
    print(f"\n Análise das Bounding Boxes")    
    
    # Compilar todas as bounding boxes
    all_bboxes = []
    for split_name, annotations in all_annotations.items():
        for ann in annotations:
            bbox_info = {
                'split': split_name,
                'width': ann['bbox'][2],
                'height': ann['bbox'][3],
                'area': ann['area'],
                'aspect_ratio': ann['bbox'][2] / ann['bbox'][3],
                'binary_class': ann['binary_class']
            }
            all_bboxes.append(bbox_info)
    
    df_bbox = pd.DataFrame(all_bboxes)
    
    # Estatísticas gerais
    print(f" Estatísticas Gerais:")
    print(f"   • Total de bounding boxes: {len(df_bbox)}")
    print(f"   • Width  - Min: {df_bbox['width'].min():.1f}, Max: {df_bbox['width'].max():.1f}, Média: {df_bbox['width'].mean():.1f}")
    print(f"   • Height - Min: {df_bbox['height'].min():.1f}, Max: {df_bbox['height'].max():.1f}, Média: {df_bbox['height'].mean():.1f}")
    print(f"   • Area   - Min: {df_bbox['area'].min():.0f}, Max: {df_bbox['area'].max():.0f}, Média: {df_bbox['area'].mean():.0f}")
    print(f"   • Aspect Ratio - Min: {df_bbox['aspect_ratio'].min():.2f}, Max: {df_bbox['aspect_ratio'].max():.2f}, Média: {df_bbox['aspect_ratio'].mean():.2f}")
    
    # Por classe
    print(f"\n POR CLASSE:")
    for cls in df_bbox['binary_class'].unique():
        subset = df_bbox[df_bbox['binary_class'] == cls]
        print(f"   {cls} ({len(subset)} boxes):")
        print(f"     • Width: {subset['width'].mean():.1f} ± {subset['width'].std():.1f}")
        print(f"     • Height: {subset['height'].mean():.1f} ± {subset['height'].std():.1f}")
        print(f"     • Area: {subset['area'].mean():.0f} ± {subset['area'].std():.0f}")
    
    # Detecção de outliers e filtros automáticos
    q1_area = df_bbox['area'].quantile(0.25)
    q3_area = df_bbox['area'].quantile(0.75)
    iqr = q3_area - q1_area
    lower_bound = q1_area - 1.5 * iqr
    upper_bound = q3_area + 1.5 * iqr
    
    outliers = df_bbox[(df_bbox['area'] < lower_bound) | (df_bbox['area'] > upper_bound)]
    print(f"\n Detecção de Outliers:")
    print(f"   • Outliers por área: {len(outliers)} ({len(outliers)/len(df_bbox)*100:.1f}%)")
    
    # Filtros automáticos baseados em percentis
    min_area_rec = df_bbox['area'].quantile(0.05)  # 5% menores
    max_area_rec = df_bbox['area'].quantile(0.95)  # 5% maiores
    min_aspect_rec = df_bbox['aspect_ratio'].quantile(0.05)
    max_aspect_rec = df_bbox['aspect_ratio'].quantile(0.95)
    
    quality_filters = {
        'min_area': min_area_rec,
        'max_area': max_area_rec,
        'min_aspect': max(0.3, min_aspect_rec),  # Não muito estreito
        'max_aspect': min(3.0, max_aspect_rec)   # Não muito alongado
    }
    
    print(f"\n Filtros de Qualidade:")
    for key, value in quality_filters.items():
        print(f"   • {key}: {value:.1f}")
    
    # Visualizações
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    colors = {'COM_corante': 'blue', 'SEM_corante': 'red'}
    
    # 1. Distribuição de áreas
    for cls in df_bbox['binary_class'].unique():
        subset = df_bbox[df_bbox['binary_class'] == cls]
        axes[0,0].hist(subset['area'], alpha=0.6, label=cls, 
                       bins=30, color=colors[cls])
    axes[0,0].axvline(min_area_rec, color='green', linestyle='--', alpha=0.7, label='Min recomendado')
    axes[0,0].axvline(max_area_rec, color='green', linestyle='--', alpha=0.7, label='Max recomendado')
    axes[0,0].set_title('Distribuição de Áreas')
    axes[0,0].set_xlabel('Área (pixels²)')
    axes[0,0].legend()
    
    # 2. Width vs Height scatter
    for cls in df_bbox['binary_class'].unique():
        subset = df_bbox[df_bbox['binary_class'] == cls]
        axes[0,1].scatter(subset['width'], subset['height'], 
                         alpha=0.6, label=cls, color=colors[cls])
    axes[0,1].plot([0, 200], [0, 200], 'k--', alpha=0.5, label='Quadrado perfeito')
    axes[0,1].set_title('Width vs Height')
    axes[0,1].set_xlabel('Width (pixels)')
    axes[0,1].set_ylabel('Height (pixels)')
    axes[0,1].legend()
    
    # 3. Aspect ratio
    for cls in df_bbox['binary_class'].unique():
        subset = df_bbox[df_bbox['binary_class'] == cls]
        axes[0,2].hist(subset['aspect_ratio'], alpha=0.6, label=cls, 
                       bins=20, color=colors[cls])
    axes[0,2].axvline(quality_filters['min_aspect'], color='green', linestyle='--', alpha=0.7)
    axes[0,2].axvline(quality_filters['max_aspect'], color='green', linestyle='--', alpha=0.7)
    axes[0,2].set_title('Distribuição de Aspect Ratio')
    axes[0,2].set_xlabel('Aspect Ratio (W/H)')
    axes[0,2].legend()
    
    # 4. Boxplot de áreas por classe
    data_for_box = [df_bbox[df_bbox['binary_class'] == cls]['area'] for cls in df_bbox['binary_class'].unique()]
    axes[1,0].boxplot(data_for_box, labels=df_bbox['binary_class'].unique())
    axes[1,0].set_title('Boxplot - Áreas por Classe')
    axes[1,0].set_ylabel('Área (pixels²)')
    
    # 5. Distribuição por split
    split_counts = df_bbox['split'].value_counts()
    axes[1,1].bar(split_counts.index, split_counts.values, alpha=0.7)
    axes[1,1].set_title('Distribuição por Split')
    axes[1,1].set_ylabel('Número de Bounding Boxes')
    
    # 6. Área vs Aspect Ratio
    scatter = axes[1,2].scatter(df_bbox['area'], df_bbox['aspect_ratio'], 
                               c=[colors[cls] for cls in df_bbox['binary_class']], 
                               alpha=0.6)
    axes[1,2].set_title('Área vs Aspect Ratio')
    axes[1,2].set_xlabel('Área (pixels²)')
    axes[1,2].set_ylabel('Aspect Ratio')
    
    plt.tight_layout()
    plt.show()
    
    return df_bbox, quality_filters


# 4. Conversão e Extração


def copy_images_for_yolo(source_images_path, target_images_path, images_dict):
    
    copied_count = 0
    failed_count = 0
    
    for image_info in images_dict.values():
        source_path = Path(source_images_path) / image_info['file_name']
        target_path = Path(target_images_path) / image_info['file_name']
        
        try:
            if source_path.exists():
                shutil.copy2(source_path, target_path)
                copied_count += 1
            else:
                failed_count += 1
        except Exception as e:
            failed_count += 1
    
    return copied_count, failed_count

def convert_and_extract_unified(annotations_list, images_dict, images_path, 
                               yolo_labels_path, patches_base_path, split_name, 
                               patch_size, quality_filters):
    
    print(f"    Processando {split_name}...")
    
    # Contadores
    yolo_annotations = 0
    patches_extracted = 0
    filtered_count = 0
    class_counts = defaultdict(int)
    
    # Agrupar anotações por imagem para processar uma vez
    annotations_by_image = defaultdict(list)
    for ann in annotations_list:
        annotations_by_image[ann['image_id']].append(ann)
    
    for image_id, image_annotations in annotations_by_image.items():
        if image_id not in images_dict:
            continue
            
        image_info = images_dict[image_id]
        image_width = image_info['width']
        image_height = image_info['height']
        
        # Carregar imagem uma única vez
        image_path = Path(images_path) / image_info['file_name']
        if not image_path.exists():
            continue
            
        image = cv2.imread(str(image_path))
        if image is None:
            continue
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Filtrar anotações válidas
        valid_annotations = []
        yolo_lines = []
        
        for idx, ann in enumerate(image_annotations):
            bbox = ann['bbox']
            area = ann['area']
            aspect_ratio = bbox[2] / bbox[3]
            binary_class = ann['binary_class']
            
            # Aplicar filtros de qualidade
            if not (area >= quality_filters['min_area'] and 
                   area <= quality_filters['max_area'] and
                   aspect_ratio >= quality_filters['min_aspect'] and 
                   aspect_ratio <= quality_filters['max_aspect']):
                filtered_count += 1
                continue
            
            valid_annotations.append((ann, idx))
            
            # Preparar linha YOLO
            x_center = (bbox[0] + bbox[2] / 2) / image_width
            y_center = (bbox[1] + bbox[3] / 2) / image_height
            width_norm = bbox[2] / image_width
            height_norm = bbox[3] / image_height
            class_id = CLASS_MAPPING_REVERSE[binary_class]
            
            yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width_norm:.6f} {height_norm:.6f}")
        
        # Se não há anotações válidas, pular esta imagem
        if not valid_annotations:
            continue
        
        # Salvar arquivo YOLO
        yolo_filename = Path(image_info['file_name']).stem + '.txt'
        yolo_path = yolo_labels_path / yolo_filename
        
        with open(yolo_path, 'w') as f:
            for line in yolo_lines:
                f.write(line + '\n')
                yolo_annotations += 1
        
        # Extrair patches
        for ann, idx in valid_annotations:
            bbox = ann['bbox']
            binary_class = ann['binary_class']
            
            # Extrair patch
            x, y, w, h = map(int, bbox)
            
            # Garantir bounds
            x1 = max(0, x)
            y1 = max(0, y)
            x2 = min(image_rgb.shape[1], x + w)
            y2 = min(image_rgb.shape[0], y + h)
            
            if x2 <= x1 or y2 <= y1:
                continue
            
            # Extrair e redimensionar patch
            patch = image_rgb[y1:y2, x1:x2]
            patch_resized = cv2.resize(patch, (patch_size, patch_size))
            
            # Salvar patch
            image_basename = Path(image_info['file_name']).stem
            patch_filename = f"{image_basename}_{idx:03d}_{binary_class}.jpg"
            patch_output_dir = patches_base_path / split_name / binary_class
            patch_path = patch_output_dir / patch_filename
            
            # Converter RGB para BGR para cv2.imwrite
            patch_bgr = cv2.cvtColor(patch_resized, cv2.COLOR_RGB2BGR)
            cv2.imwrite(str(patch_path), patch_bgr)
            
            patches_extracted += 1
            class_counts[binary_class] += 1
    
    return yolo_annotations, patches_extracted, filtered_count, dict(class_counts)


# 5. Função Principal


def main_data_preparation():
    
    print(f"\n Iniciando Preparação Completa dos Dados")    
        
    # Etapa 1: Visualização e Análise
    print(f"\n Etapa 1: Visualização e Análise")    
    
    # Visualizar amostras de cada split
    sample_results = {}
    for split_name in ['train', 'valid', 'test']:
        if split_name in all_annotations and len(all_annotations[split_name]) > 0:
            json_path = PATHS[split_name]['json']
            images_path = PATHS[split_name]['images']
            
            with open(json_path, 'r') as f:
                coco_data = json.load(f)
            images_dict = {img['id']: img for img in coco_data['images']}
            
            sample_ids = visualize_annotations_sample(
                split_name, images_dict, all_annotations[split_name], 
                images_path, n_samples=6
            )
            sample_results[split_name] = sample_ids
    
    # Análise detalhada das bounding boxes
    df_bbox, quality_filters = analyze_bbox_detailed(all_annotations)
    
    print(f"\n Filtros Definidos:")
    for key, value in quality_filters.items():
        print(f"   • {key}: {value:.1f}")
    
    # Etapa 2: Conversão e Extração
    print(f"\n Etapa 2: Conversão e Extração")    
    
    # Criar diretórios
    directories = create_directories()
    
    # Processar cada split
    total_stats = {
        'yolo_annotations': 0,
        'patches_extracted': 0,
        'filtered_total': 0,
        'class_counts': defaultdict(int)
    }
    
    for split_name in ['train', 'valid', 'test']:
        if split_name not in all_annotations:
            print(f" Split {split_name} não encontrado, pulando...")
            continue
            
        print(f"\n Processando Split: {split_name.upper()}")
        
        # Carregar dados COCO do split
        json_path = PATHS[split_name]['json']
        images_path = PATHS[split_name]['images']
        
        with open(json_path, 'r') as f:
            coco_data = json.load(f)
        images_dict = {img['id']: img for img in coco_data['images']}
        annotations_list = all_annotations[split_name]
        
        print(f"    Dados originais: {len(images_dict)} imagens, {len(annotations_list)} anotações")
        
        # Copiar imagens para YOLO
        yolo_images_path = directories['yolo_dataset'] / "images" / split_name
        copied, failed = copy_images_for_yolo(images_path, yolo_images_path, images_dict)
        print(f"    Imagens copiadas: {copied}, Falhas: {failed}")
        
        # Conversão e extração unificada
        yolo_labels_path = directories['yolo_dataset'] / "labels" / split_name
        
        yolo_ann, patches_ext, filtered, class_counts = convert_and_extract_unified(
            annotations_list, images_dict, images_path,
            yolo_labels_path, directories['patches_dataset'], split_name,
            PATCH_SIZE, quality_filters
        )
        
        print(f"    YOLO - Anotações: {yolo_ann}")
        print(f"    PATCHES - Extraídos: {patches_ext}, Filtrados: {filtered}")
        for class_name, count in class_counts.items():
            print(f"      • {class_name}: {count} patches")
        
        # Atualizar estatísticas totais
        total_stats['yolo_annotations'] += yolo_ann
        total_stats['patches_extracted'] += patches_ext
        total_stats['filtered_total'] += filtered
        for class_name, count in class_counts.items():
            total_stats['class_counts'][class_name] += count
    
    # Etapa 3: Finalização
    print(f"\n Etapa 3: Finalização")    
    
    # Criar arquivo YAML para YOLO
    yaml_path = create_yolo_dataset_yaml(directories['yolo_dataset'], CLASS_NAMES)
    print(f" Dataset YAML criado: {yaml_path}")
    
    # Relatório Final
    print(f"\n")
    print(" Relatório Final de Preparação de Dados")    
    
    print(f"\n Dataset YOLO:")
    print(f"   • Anotações processadas: {total_stats['yolo_annotations']}")
    print(f"   • Localização: {directories['yolo_dataset']}")
    
    print(f"\n Dataset Patches:")
    print(f"   • Patches extraídos: {total_stats['patches_extracted']}")
    print(f"   • Patches filtrados: {total_stats['filtered_total']}")
    
    if total_stats['patches_extracted'] + total_stats['filtered_total'] > 0:
        aproveitamento = total_stats['patches_extracted'] / (total_stats['patches_extracted'] + total_stats['filtered_total']) * 100
        print(f"   • Taxa de aproveitamento: {aproveitamento:.1f}%")
    
    total_patches = sum(total_stats['class_counts'].values())
    for class_name, count in total_stats['class_counts'].items():
        if total_patches > 0:
            pct = (count / total_patches) * 100
            print(f"   • {class_name}: {count} patches ({pct:.1f}%)")
    
    print(f"   • Localização: {directories['patches_dataset']}")
    
    # Verificação de balanceamento
    if total_stats['class_counts']:
        com_patches = total_stats['class_counts']['COM_corante']
        sem_patches = total_stats['class_counts']['SEM_corante']
        if com_patches > 0 and sem_patches > 0:
            balance_ratio = min(com_patches, sem_patches) / max(com_patches, sem_patches)
            
            print(f"\n Balanceamento:")
            print(f"   • Ratio de balanceamento: {balance_ratio:.3f}")
            if balance_ratio > 0.8:
                print("   • Status: Excelente balanceamento")
            elif balance_ratio > 0.6:
                print("   • Status: Balanceamento aceitável")
            else:
                print("   • Status: Desbalanceamento detectado")
    
    print(f"\n")
    print(" Bloco 2 Concluído")
    print(" Datasets YOLO e Patches prontos para treinamento")    
    
    return True, directories, quality_filters, total_stats

# 6. Execução

if __name__ == "__main__":
    # Executar preparação completa dos dados
    success, directories, filters, stats = main_data_preparation()
    
    if success:        
        print("   Dados organizados e filtrados")        
        
        # Salvar informações importantes para os próximos blocos
        preparation_info = {
            'directories': {str(k): str(v) for k, v in directories.items()},
            'quality_filters': filters,
            'statistics': dict(stats),
            'patch_size': PATCH_SIZE,
            'class_names': CLASS_NAMES
        }
        
        # Salvar em arquivo JSON para uso posterior
        info_path = Path(OUTPUT_BASE) / "preparation_info.json"
        with open(info_path, 'w') as f:
            json.dump(preparation_info, f, indent=2, default=str)
        
        print(f"   Informações salvas em: {info_path}")
    else:
        print(f"\n  Execute os blocos anteriores primeiro!")

In [ ]:
# Bloco 3: Treinamento do Detector YOLO
# Pipeline: YOLO (Detecção e Classificação)

print(" BLOCO 3: Treinamento do Detector YOLO")
print(" Objetivo: Fine-tuning YOLOv8 para detecção de células")

# 1. Instalação e Setup do Ultralytics YOLO

def install_and_setup_yolo():
    
    print(f"\n Setup do Ambiente YOLO")        
    
    # Verificar GPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"    Dispositivo: {device}")
    
    if device == 'cuda':
        print(f"    GPU: {torch.cuda.get_device_name(0)}")
        print(f"    VRAM disponível: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    return YOLO, device

# 2. Configuração de Treinamento

# Configurações otimizadas para GPU P100 do Kaggle
TRAINING_CONFIG = {
    'model_size': 'yolov8s.pt',  # Small - balanceado para P100
    'epochs': 100,               # Máximo, com early stopping
    'batch_size': 16,            # Otimizado para P100
    'imgsz': 640,                # Tamanho padrão das imagens
    'patience': 15,              # Early stopping
    'save_period': 10,           # Salvar checkpoint a cada 10 epochs
    'optimizer': 'AdamW',        # Otimizador moderno
    'lr0': 0.01,                 # Learning rate inicial
    'lrf': 0.1,                  # Learning rate final (decay)
    'momentum': 0.937,           # Momentum para SGD
    'weight_decay': 0.0005,      # Regularização
    'warmup_epochs': 3,          # Epochs de warmup
    'warmup_momentum': 0.8,      # Momentum durante warmup
    'box': 7.5,                  # Box loss gain
    'cls': 0.5,                  # Class loss gain
    'dfl': 1.5,                  # DFL loss gain
    'mixup': 0.0,                # Mixup augmentation (off para microscopia)
    'copy_paste': 0.0,           # Copy paste augmentation (off)
    'degrees': 10.0,             # Rotação máxima (graus)
    'translate': 0.1,            # Translação máxima (fração)
    'scale': 0.5,                # Scale variation (+/- gain)
    'shear': 2.0,                # Shear máximo (graus)
    'perspective': 0.0,          # Perspective transform (off para microscopia)
    'flipud': 0.0,               # Flip vertical probability (off)
    'fliplr': 0.5,               # Flip horizontal probability
    'mosaic': 0.0,               # Mosaic augmentation (off para microscopia)
    'hsv_h': 0.015,              # Hue augmentation (small para microscopia)
    'hsv_s': 0.7,                # Saturation augmentation
    'hsv_v': 0.4                 # Value augmentation
}

DATASET_PATH = "/kaggle/working/yolo_dataset"
OUTPUT_PATH = "/kaggle/working/yolo_training"

print(f" Configuração de Treinamento:")
print(f"   • Modelo: {TRAINING_CONFIG['model_size']}")
print(f"   • Epochs máximos: {TRAINING_CONFIG['epochs']}")
print(f"   • Batch size: {TRAINING_CONFIG['batch_size']}")
print(f"   • Early stopping: {TRAINING_CONFIG['patience']} epochs")
print(f"   • Dataset: {DATASET_PATH}")
print(f"   • Saída: {OUTPUT_PATH}")

# 3. Funções de Monitoramento e Análise

def setup_training_monitoring():
    
    # Criar diretório de saída
    Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)
    
    # Configurar logging
    training_log = {
        'start_time': time.time(),
        'epochs_completed': 0,
        'best_map50': 0.0,
        'best_map50_95': 0.0,
        'training_metrics': [],
        'validation_metrics': [],
        'overfitting_alerts': []
    }
    
    return training_log

def analyze_training_progress(results_path):
    
    print(f"\n Analisando Progresso do Treinamento")    
    
    # Carregar resultados
    results_file = Path(results_path) / "results.csv" 
    
    
    df = pd.read_csv(results_file)
    
    # Renomear colunas com espaços em branco
    df.columns = df.columns.str.strip()
    
    print(f"    Epochs processados: {len(df)}")
    
    # Métricas principais
    if len(df) > 0:
        latest = df.iloc[-1]
        print(f"    Último epoch:")
        print(f"      • Train Loss: {latest.get('train/box_loss', 0):.4f}")
        print(f"      • Val Loss: {latest.get('val/box_loss', 0):.4f}")
        print(f"      • mAP@0.5: {latest.get('metrics/mAP50(B)', 0):.4f}")
        print(f"      • mAP@0.5:0.95: {latest.get('metrics/mAP50-95(B)', 0):.4f}")
        
        # Detectar overfitting (últimos 5 epochs)
        if len(df) >= 10:
            recent_train = df['train/box_loss'].tail(5).mean()
            recent_val = df['val/box_loss'].tail(5).mean()
            overfitting_ratio = recent_val / recent_train
            
            print(f"    Análise de Overfitting:")
            print(f"      • Ratio Val/Train Loss: {overfitting_ratio:.3f}")
            
            if overfitting_ratio > 1.5:
                print("       ALERTA: Possível overfitting detectado!")
            elif overfitting_ratio > 1.2:
                print("       Monitore: Val loss começando a divergir")
            else:
                print("       Sem sinais de overfitting")
    
    return df
        
    

def plot_training_curves(results_df, output_path):
    
    print(f"    Gerando gráficos de treinamento...")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    epochs = range(len(results_df))
    
    # 1. Box Loss
    if 'train/box_loss' in results_df.columns and 'val/box_loss' in results_df.columns:
        axes[0,0].plot(epochs, results_df['train/box_loss'], label='Train', color='blue')
        axes[0,0].plot(epochs, results_df['val/box_loss'], label='Validation', color='red')
        axes[0,0].set_title('Box Loss')
        axes[0,0].set_xlabel('Epoch')
        axes[0,0].set_ylabel('Loss')
        axes[0,0].legend()
        axes[0,0].grid(True, alpha=0.3)
    
    # 2. Class Loss
    if 'train/cls_loss' in results_df.columns and 'val/cls_loss' in results_df.columns:
        axes[0,1].plot(epochs, results_df['train/cls_loss'], label='Train', color='blue')
        axes[0,1].plot(epochs, results_df['val/cls_loss'], label='Validation', color='red')
        axes[0,1].set_title('Classification Loss')
        axes[0,1].set_xlabel('Epoch')
        axes[0,1].set_ylabel('Loss')
        axes[0,1].legend()
        axes[0,1].grid(True, alpha=0.3)
    
    # 3. mAP Metrics
    if 'metrics/mAP50(B)' in results_df.columns:
        axes[1,0].plot(epochs, results_df['metrics/mAP50(B)'], label='mAP@0.5', color='green')
        if 'metrics/mAP50-95(B)' in results_df.columns:
            axes[1,0].plot(epochs, results_df['metrics/mAP50-95(B)'], label='mAP@0.5:0.95', color='orange')
        axes[1,0].set_title('mAP Metrics')
        axes[1,0].set_xlabel('Epoch')
        axes[1,0].set_ylabel('mAP')
        axes[1,0].legend()
        axes[1,0].grid(True, alpha=0.3)
    
    # 4. Precision/Recall
    if 'metrics/precision(B)' in results_df.columns and 'metrics/recall(B)' in results_df.columns:
        axes[1,1].plot(epochs, results_df['metrics/precision(B)'], label='Precision', color='purple')
        axes[1,1].plot(epochs, results_df['metrics/recall(B)'], label='Recall', color='brown')
        axes[1,1].set_title('Precision & Recall')
        axes[1,1].set_xlabel('Epoch')
        axes[1,1].set_ylabel('Score')
        axes[1,1].legend()
        axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Salvar gráfico
    plot_path = Path(output_path) / "training_curves.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"    Gráficos salvos em: {plot_path}")

def validate_dataset_yaml():
   
    yaml_path = Path(DATASET_PATH) / "dataset.yaml"    
       
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    
    print(f" Dataset YAML validado:")
    print(f"   • Path: {data.get('path')}")
    print(f"   • Classes: {data.get('nc')} - {data.get('names')}")
    print(f"   • Splits: train, val, test")    

# 4. Função Principal de Treinamento

def train_yolo_detector():
   
    print(f"\n Iniciando Treinamento do Detector YOLO")    
    
    # 1. Setup inicial
    YOLO, device = install_and_setup_yolo()
            
    # 2. Setup de monitoramento
    training_log = setup_training_monitoring()
    
    # 3. Carregar modelo pré-treinado
    print(f"\n Carregamento Modelo Pré-Treinado")        
    
    model = YOLO(TRAINING_CONFIG['model_size'])
    print(f"    Modelo {TRAINING_CONFIG['model_size']} carregado")
    print(f"    Parâmetros: {sum(p.numel() for p in model.model.parameters()):,}")   
    
    # 4. Configurar argumentos de treinamento
    train_args = {
        'data': str(Path(DATASET_PATH) / "dataset.yaml"),
        'epochs': TRAINING_CONFIG['epochs'],
        'batch': TRAINING_CONFIG['batch_size'],
        'imgsz': TRAINING_CONFIG['imgsz'],
        'device': device,
        'project': OUTPUT_PATH,
        'name': 'cell_detector_v1',
        'patience': TRAINING_CONFIG['patience'],
        'save_period': TRAINING_CONFIG['save_period'],
        'optimizer': TRAINING_CONFIG['optimizer'],
        'lr0': TRAINING_CONFIG['lr0'],
        'lrf': TRAINING_CONFIG['lrf'],
        'momentum': TRAINING_CONFIG['momentum'],
        'weight_decay': TRAINING_CONFIG['weight_decay'],
        'warmup_epochs': TRAINING_CONFIG['warmup_epochs'],
        'warmup_momentum': TRAINING_CONFIG['warmup_momentum'],
        'box': TRAINING_CONFIG['box'],
        'cls': TRAINING_CONFIG['cls'],
        'dfl': TRAINING_CONFIG['dfl'],
        'degrees': TRAINING_CONFIG['degrees'],
        'translate': TRAINING_CONFIG['translate'],
        'scale': TRAINING_CONFIG['scale'],
        'shear': TRAINING_CONFIG['shear'],
        'perspective': TRAINING_CONFIG['perspective'],
        'flipud': TRAINING_CONFIG['flipud'],
        'fliplr': TRAINING_CONFIG['fliplr'],
        'mosaic': TRAINING_CONFIG['mosaic'],
        'mixup': TRAINING_CONFIG['mixup'],
        'copy_paste': TRAINING_CONFIG['copy_paste'],
        'hsv_h': TRAINING_CONFIG['hsv_h'],
        'hsv_s': TRAINING_CONFIG['hsv_s'],
        'hsv_v': TRAINING_CONFIG['hsv_v'],
        'verbose': True,
        'seed': 42,  # Reprodutibilidade
        'deterministic': True,
        'single_cls': False,
        'rect': False,
        'cos_lr': True,  # Cosine learning rate scheduler
        'close_mosaic': 10,  # Disable mosaic in last N epochs
        'resume': False,
        'amp': True,  # Automatic Mixed Precision
        'fraction': 1.0,  # Dataset fraction to use
        'profile': False,
        'freeze': None,  # Freeze layers
        'multi_scale': False,  # Multi-scale training
        'overlap_mask': True,
        'mask_ratio': 4,
        'dropout': 0.0,
        'val': True,  # Validate during training
        'plots': True,  # Generate plots
        'save_json': True,  # Save results in JSON format
        'save_hybrid': False,
        'conf': None,  # Confidence threshold for predictions
        'iou': 0.7,  # IoU threshold for NMS
        'max_det': 300,  # Maximum detections per image
        'half': False,  # Use half precision
        'dnn': False,  # Use OpenCV DNN backend
        'cache': False,  # Use dataset caching
        'rect': False,  # Rectangular training
        'save_txt': False,  # Save results as txt
        'save_conf': False,  # Save confidences in txt
        'save_crop': False,  # Save cropped prediction boxes
        'show_labels': True,  # Show labels in plots
        'show_conf': True,  # Show confidences in plots
        'visualize': False,  # Visualize model predictions
        'augment': False,  # Apply augmentation during validation
        'agnostic_nms': False,  # Class-agnostic NMS
        'retina_masks': False,  # Use high resolution masks
        'boxes': True  # Show boxes in segmentation predictions
    }
    
    print(f"\n Configuração de Treinamento:")
    print(f"   • Dataset: {train_args['data']}")
    print(f"   • Epochs: {train_args['epochs']} (early stop: {train_args['patience']})")
    print(f"   • Batch size: {train_args['batch']}")
    print(f"   • Image size: {train_args['imgsz']}")
    print(f"   • Device: {train_args['device']}")
    print(f"   • Optimizer: {train_args['optimizer']}")
    print(f"   • Learning rate: {train_args['lr0']} → {train_args['lrf']}")
    
    # 5. Iniciar treinamento
    print(f"\n Iniciando Treinamento...")    
    print(f" Monitoramento: Execute analyze_training_progress() em paralelo")
    
    
    try:
        # Registrar início
        training_log['start_time'] = time.time()
        
        # TREINAR MODELO
        results = model.train(**train_args)
        
        # Registrar fim
        training_duration = time.time() - training_log['start_time']
        
        print(f"\n TREINAMENTO CONCLUÍDO!")
        
        print(f" Duração total: {training_duration/60:.1f} minutos")
        
        # Analisar resultados finais
        results_path = Path(OUTPUT_PATH) / "cell_detector_v2"
        final_results = analyze_training_progress(results_path)
        
        if final_results is not None:
            plot_training_curves(final_results, results_path)
        
        # Informações do modelo final - Corrigir path real
        actual_results_path = Path(OUTPUT_PATH) / "cell_detector_v2"  # Path real gerado
        best_model_path = actual_results_path / "weights" / "best.pt"
        last_model_path = actual_results_path / "weights" / "last.pt"
        
        print(f"\n MODELOS SALVOS:")
        print(f"    Melhor modelo: {best_model_path}")
        print(f"    Último modelo: {last_model_path}")
        
        # Métricas finais
        if final_results is not None and len(final_results) > 0:
            best_map50 = final_results['metrics/mAP50(B)'].max()
            best_map50_95 = final_results['metrics/mAP50-95(B)'].max()
            
            print(f"\n MELHORES MÉTRICAS:")
            print(f"   • mAP@0.5: {best_map50:.4f}")
            print(f"   • mAP@0.5:0.95: {best_map50_95:.4f}")
        
        print(f"\n BLOCO 3 CONCLUÍDO COM SUCESSO!")
        print("    Próximo: BLOCO 4 - Modelos de Classificação")
        
        return True, results_path
        
    except Exception as e:
        print(f"\n ERRO durante treinamento: {e}")
        print("   Verifique logs para mais detalhes")
        return False, None

# 5. Execução

if __name__ == "__main__":
       
    print(" Para executar:")
    print("   success, model_path = train_yolo_detector()")

In [ ]:
success, model_path = train_yolo_detector()

In [ ]:
# Visualização do treino em outra célula (opcional):
results_df = analyze_training_progress("/kaggle/working/yolo_training/cell_detector_v1") #atenção para pasta que foi salva

In [ ]:
# Visualização de imagens individuais do treino
img = Image.open('/kaggle/working/yolo_training/cell_detector_v1/F1_curve.png')
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
# Bloco 4: Visualização das Detecções YOLO
# Análise visual do detector treinado

print(" Objetivo: Analisar visualmente as detecções do modelo treinado")

# 1. Configuração e Carregamento do Modelo

# Paths
MODEL_PATH = "/kaggle/working/yolo_training/cell_detector_v1/weights/best.pt" #atenção para pasta que foi salva
DATASET_PATH = "/kaggle/working/yolo_dataset"
TEST_IMAGES_PATH = f"{DATASET_PATH}/images/test"

# Classes
CLASS_NAMES = ['COM_corante', 'SEM_corante']
CLASS_COLORS = {'COM_corante': 'blue', 'SEM_corante': 'red'}

def load_trained_model():
    
    print(f"\n Carregando Modelo Treinado")    
    
    
    from ultralytics import YOLO
    
    # Verificar se modelo existe
    if not Path(MODEL_PATH).exists():
        print(f" ERRO: Modelo não encontrado em {MODEL_PATH}")
        return None
    
    # Carregar modelo
    model = YOLO(MODEL_PATH)
    print(f"    Modelo carregado: {MODEL_PATH}")
    print(f"    Classes: {model.names}")
    
    return model    

def get_test_images(n_samples=8):
    
    print(f"\n Selecionando Imagens de Teste")    
    
    test_images_dir = Path(TEST_IMAGES_PATH)    
        
    # Listar imagens
    image_files = list(test_images_dir.glob("*.jpg")) + list(test_images_dir.glob("*.jpeg")) + list(test_images_dir.glob("*.png"))
    
    if len(image_files) == 0:
        print(f" ERRO: Nenhuma imagem encontrada em {test_images_dir}")
        return []
    
    print(f"    Total de imagens de teste: {len(image_files)}")
    
    # Selecionar amostra aleatória
    random.seed(42)  # Reprodutibilidade
    if len(image_files) < n_samples:
        n_samples = len(image_files)
    
    selected_images = random.sample(image_files, n_samples)
    
    print(f"    Selecionadas: {len(selected_images)} imagens")
    for img in selected_images:
        print(f"      • {img.name}")
    
    return selected_images

# 2. Funções de Visualização

def predict_and_visualize(model, image_path, conf_threshold=0.25):    
    
    # Carregar imagem
    image = cv2.imread(str(image_path))
    if image is None:
        print(f" Erro ao carregar: {image_path}")
        return None, None
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Predição
    results = model(image_path, conf=conf_threshold, verbose=False)
    
    # Extrair informações das detecções
    detections = []
    if len(results) > 0 and results[0].boxes is not None:
        boxes = results[0].boxes
        
        for i in range(len(boxes)):
            box = boxes.xyxy[i].cpu().numpy()  # [x1, y1, x2, y2]
            conf = boxes.conf[i].cpu().numpy()
            cls = int(boxes.cls[i].cpu().numpy())
            
            detection = {
                'bbox': box,
                'confidence': conf,
                'class_id': cls,
                'class_name': CLASS_NAMES[cls]
            }
            detections.append(detection)
    
    return image_rgb, detections 
    
def create_visualization_grid(model, image_paths, conf_threshold=0.25):
    
    print(f"\n Criando Visualizações")    
    
    n_images = len(image_paths)    
    
    # Calcular grid
    n_cols = min(4, n_images)
    n_rows = (n_images + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
    if n_rows == 1:
        axes = [axes] if n_cols == 1 else axes
    else:
        axes = axes.flatten()
    
    total_detections = 0
    class_counts = {'COM_corante': 0, 'SEM_corante': 0}
    
    for idx, image_path in enumerate(image_paths):
        print(f"    Processando: {image_path.name}")
        
        # Predição
        image, detections = predict_and_visualize(model, image_path, conf_threshold)        
        
        # Plotar imagem
        axes[idx].imshow(image)
        
        # Plotar detecções
        detection_count = len(detections)
        total_detections += detection_count
        
        for det in detections:
            bbox = det['bbox']
            conf = det['confidence']
            class_name = det['class_name']
            class_counts[class_name] += 1
            
            # Criar retângulo
            x1, y1, x2, y2 = bbox
            width = x2 - x1
            height = y2 - y1
            
            rect = patches.Rectangle(
                (x1, y1), width, height,
                linewidth=2, 
                edgecolor=CLASS_COLORS[class_name], 
                facecolor='none', 
                alpha=0.8
            )
            axes[idx].add_patch(rect)
            
            # Label com confiança
            label = f'{class_name}\n{conf:.2f}' #editar para mostrar valor de confiança
            axes[idx].text(x1, y1-10, label, 
                          color=CLASS_COLORS[class_name], 
                          fontsize=8, 
                          fontweight='bold',
                          bbox=dict(boxstyle="round,pad=0.3", 
                                   facecolor='white', 
                                   alpha=0.8))
        
        # Título com estatísticas
        title = f'{image_path.name}\n{detection_count} células detectadas'
        axes[idx].set_title(title, fontsize=10)
        axes[idx].axis('off')
    
    # Ocultar axes vazios
    for idx in range(len(image_paths), len(axes)):
        axes[idx].axis('off')
    
    # Título geral
    main_title = f'Detecções YOLO - Confidence ≥ {conf_threshold}\n'
    main_title += f'Total: {total_detections} células | '
    main_title += f'COM_corante: {class_counts["COM_corante"]} | '
    main_title += f'SEM_corante: {class_counts["SEM_corante"]}'
    
    plt.suptitle(main_title, fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
    
    return total_detections, class_counts

def analyze_detections_statistics(model, image_paths, conf_threshold=0.25):
    
    print(f"\n Análise das Detecções")    
    
    all_detections = []
    all_confidences = {'COM_corante': [], 'SEM_corante': []}
    
    for image_path in image_paths:
        image, detections = predict_and_visualize(model, image_path, conf_threshold)
        
        if detections:
            for det in detections:
                all_detections.append(det)
                all_confidences[det['class_name']].append(det['confidence'])
    
    if not all_detections:
        print("    Nenhuma detecção encontrada")
        return
    
    # Estatísticas gerais
    total_detections = len(all_detections)
    class_counts = {'COM_corante': 0, 'SEM_corante': 0}
    
    for det in all_detections:
        class_counts[det['class_name']] += 1
    
    print(f"    Resumo Geral:")
    print(f"      • Total de detecções: {total_detections}")
    print(f"      • COM_corante: {class_counts['COM_corante']} ({class_counts['COM_corante']/total_detections*100:.1f}%)")
    print(f"      • SEM_corante: {class_counts['SEM_corante']} ({class_counts['SEM_corante']/total_detections*100:.1f}%)")
    
    # Estatísticas de confiança
    print(f"\n    Análise de Confiança:")
    for class_name, confidences in all_confidences.items():
        if confidences:
            mean_conf = np.mean(confidences)
            std_conf = np.std(confidences)
            min_conf = np.min(confidences)
            max_conf = np.max(confidences)
            
            print(f"      {class_name}:")
            print(f"         • Confiança média: {mean_conf:.3f} ± {std_conf:.3f}")
            print(f"         • Intervalo: [{min_conf:.3f}, {max_conf:.3f}]")
    
    # Plotar distribuição de confidências
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histograma de confidências
    for class_name, confidences in all_confidences.items():
        if confidences:
            axes[0].hist(confidences, alpha=0.7, label=class_name, 
                        color=CLASS_COLORS[class_name], bins=20)
    
    axes[0].set_title('Distribuição de Confidências')
    axes[0].set_xlabel('Confiança')
    axes[0].set_ylabel('Frequência')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Boxplot de confidências
    conf_data = [all_confidences[cls] for cls in CLASS_NAMES if all_confidences[cls]]
    conf_labels = [cls for cls in CLASS_NAMES if all_confidences[cls]]
    
    axes[1].boxplot(conf_data, labels=conf_labels)
    axes[1].set_title('Boxplot - Confidências por Classe')
    axes[1].set_ylabel('Confiança')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 3. Função Principal de Visualização

def main_visualization(conf_threshold=0.25, n_samples=8):
    
    print(f" Iniciando Visualização das Detecções")    
    
    # 1. Carregar modelo
    model = load_trained_model()    
    
    # 2. Selecionar imagens
    image_paths = get_test_images(n_samples)    
    
    # 3. Criar visualizações
    total_detections, class_counts = create_visualization_grid(
        model, image_paths, conf_threshold
    )
    
    # 4. Análise estatística
    analyze_detections_statistics(model, image_paths, conf_threshold)
    
    # 5. Resumo final
    print(f"\n Visualização Concluída")
    
    print(f"    Confiança mínima: {conf_threshold}")
    print(f"    Total de detecções: {total_detections}")
    print(f"    COM_corante: {class_counts['COM_corante']}")
    print(f"    SEM_corante: {class_counts['SEM_corante']}")    

# 4. Execução

if __name__ == "__main__":       
    print(" Para executar:")
    print("   main_visualization(conf_threshold=0.25, n_samples=8)")    
    print(" Parâmetros:")
    print("   • conf_threshold: confiança mínima (0.1-0.9)")
    print("   • n_samples: número de imagens (1-20)")        

In [ ]:
main_visualization()

In [ ]:
# Bloco 5: Otimização YOLO 
# Estratégias para melhorar performance do detector YOLO

print(" Bloco 5: Otimização do Detector YOLO")
print(" Objetivo: Melhorar o mAP@0.5 atual")
print(" Estratégias: Fine-tuning avançado + Hyperparameter tuning")

# Configurações

# Paths - AJUSTAR conforme sua estrutura
CURRENT_MODEL = "/kaggle/working/yolo_training/cell_detector_v1/weights/best.pt"
DATASET_PATH = "/kaggle/working/yolo_dataset"
OUTPUT_PATH = "/kaggle/working/yolo_optimization"

# Análise do Modelo Baseline

def analyze_current_model():
    
    print(f"\n Análise do Modelo Baseline")
    
    model = YOLO(CURRENT_MODEL)
    
    print(f"    Modelo carregado: {CURRENT_MODEL}") # Atualizar de acordo com performance do modelo
    print(f"    Performance baseline:")
    print(f"      • mAP@0.5: 90.8%")
    print(f"      • mAP@0.5:0.95: 58.0%")
    
    return model

# Estratégia 1: Fine-Tuning Avançado

def strategy_1_advanced_fine_tuning():
        
    print(f"\n Estratégia 1: Fine-Tuning Avançado")
    
    # Configuração otimizada para fine-tuning
    config = {
        'model': 'yolov8s.pt',
        'data': f"{DATASET_PATH}/dataset.yaml",
        'epochs': 100,
        'batch': 8,
        'imgsz': 640,
        'patience': 20,
        'project': OUTPUT_PATH,
        'name': 'advanced_fine_tune',
        
        # FINE-TUNING ESPECÍFICO
        'lr0': 0.0001,        # Learning rate muito baixo
        'lrf': 0.01,          # Decay menor
        'weight_decay': 0.001, # Regularização aumentada
        'dropout': 0.1,       # Dropout para evitar overfitting
        'warmup_epochs': 5,
        
        # AUGMENTATION PARA MICROSCOPIA
        'degrees': 5.0,       # Rotação reduzida
        'translate': 0.05,    # Translação mínima
        'scale': 0.2,         # Scale reduzido
        'flipud': 0.0,        # Sem flip vertical
        'fliplr': 0.3,        # Flip horizontal reduzido
        'mosaic': 0.5,        # Mosaic reduzido
        'hsv_h': 0.01,        # Preservar cores do corante
        'hsv_s': 0.3,
        'hsv_v': 0.2,
        
        # LOSS WEIGHTS
        'box': 7.5,
        'cls': 0.75,          # Foco na classificação
        
        # OTIMIZAÇÕES
        'optimizer': 'AdamW',
        'cos_lr': True,
        'amp': True,
        'seed': 42,
        'deterministic': True,
        'verbose': True
    }
    
    print(f"    Configuração:")
    print(f"      • Learning rate: {config['lr0']} (muito baixo)")
    print(f"      • Regularização: weight_decay={config['weight_decay']}, dropout={config['dropout']}")
    print(f"      • Augmentation: Conservador para microscopia")
    print(f"      • Class loss: {config['cls']} (aumentado)")
    
    print(f"\n    Iniciando fine-tuning avançado...")
    model = YOLO('yolov8s.pt')
    results = model.train(**config)
    
    print(f"     Fine-tuning avançado concluído.")
    return results, f"{OUTPUT_PATH}/advanced_fine_tune"

# Estratégia 2: Otimização de Hiperparâmetros

def strategy_2_hyperparameter_optimization():
    
    print(f"\n Estratégia 2: Otimização de Hiperparâmetros")
    
    # 3 sub-estratégias para testar
    configs = [
        {
            'name': 'high_class_weight',
            'description': 'Peso maior para classificação',
            'config': {
                'cls': 1.0,    # Foco na classificação COM vs SEM
                'box': 7.0,    # Peso menor para detecção
                'epochs': 50
            }
        },
        {
            'name': 'high_regularization', 
            'description': 'Regularização aumentada',
            'config': {
                'weight_decay': 0.001,  # Regularização forte
                'dropout': 0.2,         # Dropout alto
                'warmup_epochs': 10,    # Warmup longo
                'epochs': 50
            }
        },
        {
            'name': 'minimal_augmentation',
            'description': 'Augmentation mínimo',
            'config': {
                'degrees': 2.0,     # Rotação mínima
                'translate': 0.02,  # Translação mínima
                'scale': 0.1,       # Scale mínimo
                'mosaic': 0.2,      # Mosaic reduzido
                'mixup': 0.0,       # Sem mixup
                'epochs': 50
            }
        }
    ]
    
    results = {}
    
    for config_info in configs:
        name = config_info['name']
        description = config_info['description']
        specific_config = config_info['config']
        
        print(f"\n     Testando: {description}")
        print(f"        Config: {name}")
        
        # Configuração base comum
        base_config = {
            'data': f"{DATASET_PATH}/dataset.yaml",
            'batch': 8,
            'imgsz': 640,
            'patience': 10,
            'project': OUTPUT_PATH,
            'name': f'hyperparam_{name}',
            'seed': 42,
            'deterministic': True,
            'verbose': False
        }
        
        # Combinar configurações
        full_config = {**base_config, **specific_config}
        
        # Treinar modelo
        model = YOLO('yolov8s.pt')
        result = model.train(**full_config)
        
        results[name] = {
            'description': description,
            'config': specific_config,
            'result_path': f"{OUTPUT_PATH}/hyperparam_{name}",
            'success': True
        }
        
        print(f"         Concluído: {name}")
    
    return results

# Comparação de Resultados

def compare_optimization_results(original_map=0.908): #definir de acordo com o modelo
    
    print(f"\n Comparação de Resultados de Otimização")
    
    results_summary = []
    
    # 1. Resultado baseline
    results_summary.append({
        'Modelo': 'YOLO Original',
        'mAP@0.5': f"{original_map:.3f}",
        'Estratégia': 'Baseline',        
        'Status': 'Referência'
    })
    
    # 2. Fine-tuning avançado
    fine_tune_path = Path(f"{OUTPUT_PATH}/advanced_fine_tune")
    if fine_tune_path.exists():
        results_file = fine_tune_path / "results.csv"
        if results_file.exists():
            try:
                df = pd.read_csv(results_file)
                best_map = df['metrics/mAP50(B)'].max()
                
                results_summary.append({
                    'Modelo': 'Fine-tuning Avançado',
                    'mAP@0.5': f"{best_map:.3f}",
                    'Estratégia': 'Learning rate baixo + regularização',                    
                    'Status': 'Melhorado' if best_map > original_map else 'Similar'
                })
            except:
                print("    Erro ao ler resultados do fine-tuning")
    
    # 3. Hyperparameter optimization (3 sub-estratégias)
    hyperparam_configs = ['high_class_weight', 'high_regularization', 'minimal_augmentation']
    strategy_names = {
        'high_class_weight': 'High Class Weight',
        'high_regularization': 'High Regularization', 
        'minimal_augmentation': 'Minimal Augmentation'
    }
    
    for config_name in hyperparam_configs:
        config_path = Path(f"{OUTPUT_PATH}/hyperparam_{config_name}")
        if config_path.exists():
            results_file = config_path / "results.csv"
            if results_file.exists():
                try:
                    df = pd.read_csv(results_file)
                    best_map = df['metrics/mAP50(B)'].max()
                    
                    results_summary.append({
                        'Modelo': f'Hyperparam {config_name}',
                        'mAP@0.5': f"{best_map:.3f}",
                        'Estratégia': strategy_names[config_name],                        
                        'Status': 'Melhorado' if best_map > original_map else 'Similar'
                    })
                except:
                    print(f"     Erro ao ler resultados de {config_name}")
    
    # Criar e exibir tabela comparativa
    comparison_df = pd.DataFrame(results_summary)
    print(f"\n{comparison_df.to_string(index=False)}")
    
    # Identificar melhor resultado
    if len(results_summary) > 1:
        best_idx = 0
        best_map = original_map
        
        for i, result in enumerate(results_summary[1:], 1):
            
            current_map = float(result['mAP@0.5'])
            if current_map > best_map:
                best_map = current_map
                best_idx = i            
        
        if best_idx > 0:
            improvement = ((best_map - original_map) / original_map) * 100
            print(f"\n Melhor Resultado:")
            print(f"   • Modelo: {results_summary[best_idx]['Modelo']}")
            print(f"   • mAP@0.5: {best_map:.3f}")
            print(f"   • Melhoria: {improvement:.1f}%")
            print(f"   • Estratégia: {results_summary[best_idx]['Estratégia']}")
        else:
            print(f"\n Resultado:")
            print(f"   • Modelo original mantém melhor performance")
            print(f"   • mAP@0.5: {original_map:.3f} (baseline)")
    
    return comparison_df

# Função Principal

def optimize_yolo_detector():
    
    print(f"\n Ininciando Otimização")
    
    # Criar diretório de saída
    Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)
    
    # ETAPA 1: Análise do modelo baseline
    print(f"\n")
    print(f" Etapa 1: Análise do Modelo Baseline")
    print(f"\n")
    current_model = analyze_current_model()
    
    # ETAPA 2: Estratégia 1 - Fine-tuning avançado 
    print(f"\n")
    print(f" Etapa 2: Estratégia 1 - Fine-Tuning")
    print(f"\n")
    fine_tune_results, fine_tune_path = strategy_1_advanced_fine_tuning()
    
    # ETAPA 3: Estratégia 2 - Hiperparameter optimization
    print(f"\n")
    print(f" Etapa 3: Estratégia 2 - Hiperparameter Optimization")
    print(f"\n")
    hyperparam_results = strategy_2_hyperparameter_optimization()
    
    # ETAPA 4: Comparação final
    print(f"\n")
    print(f" Etapa 4: Comparação Final de Resultados")
    print(f"\n")
    comparison_df = compare_optimization_results()
    
    # Resultado final
    print(f"\n")
    print(f"  Otimização Finalizada!")
    print(f"\n")
    print(f"     Todas as estratégias testadas")
    print(f"     Relatório de comparação gerado")
    print(f"     Melhor modelo identificado")
    print(f"     Resultados salvos em: {OUTPUT_PATH}")
    
    return {
        'fine_tune_results': fine_tune_results,
        'hyperparam_results': hyperparam_results,
        'comparison': comparison_df,
        'output_path': OUTPUT_PATH
    }

# Execução

if __name__ == "__main__":    
    
    print(f"\n Estratégias Implementadas:")
    print(f"    Baseline: Modelo original (referência)")
    print(f"    Estratégia 1: Fine-tuning Avançado")
    print(f"    Estratégia 2: Hyperparameter Optimization")
    print(f"      ├── High Class Weight")
    print(f"      ├── High Regularization") 
    print(f"      └── Minimal Augmentation")
    print(f"    Comparação e Análise Final")
    
    print(f"\n Para Executar:")
    print(f"   results = optimize_yolo_detector()")   

In [ ]:
# Executar otimização
results = optimize_yolo_detector()

In [ ]:
# Bloco 6: Threshold Optimization para Todos os 5 Modelos
# Otimiza threshold individual para cada modelo treinado

print(" Bloco 6: Otimização de Threshold para Todos os Modelos")
print(" Objetivo: Encontrar threshold ótimo individual")
print(" Estratégia: Maximizar F1-Score por modelo")

# Configurações

# Configuração dos 5 Modelos
MODELS_CONFIG = {
    'Baseline': {
        'path': '/kaggle/working/yolo_training/cell_detector_v1/weights/best.pt', 
        'description': 'Modelo original de referência'
    },
    'Fine-tuning': {
        'path': '/kaggle/working/yolo_optimization/advanced_fine_tune/weights/best.pt', 
        'description': 'Fine-tuning com learning rate baixo'
    },
    'High Class Weight': {
        'path': '/kaggle/working/yolo_optimization/hyperparam_high_class_weight/weights/best.pt',
        'description': 'Peso aumentado para classificação'
    },
    'High Regularization': {
        'path': '/kaggle/working/yolo_optimization/hyperparam_high_regularization/weights/best.pt',
        'description': 'Regularização aumentada'
    },
    'Minimal Augmentation': {
        'path': '/kaggle/working/yolo_optimization/hyperparam_minimal_augmentation/weights/best.pt',
        'description': 'Augmentation mínimo'
    }
}

# Dataset e configurações
DATASET_PATH = "/kaggle/working/yolo_dataset"
VALIDATION_IMAGES_PATH = f"{DATASET_PATH}/images/valid"
VALIDATION_LABELS_PATH = f"{DATASET_PATH}/labels/valid"

# Classes do projeto
CLASS_NAMES = ['COM_corante', 'SEM_corante']

# Parâmetros da otimização
THRESHOLD_RANGE = np.arange(0.10, 0.90, 0.05)  # 0.10 até 0.85 com passo 0.05
IoU_THRESHOLD = 0.5  # Para cálculo de True Positive/False Positive

print(f" Configuração:")
print(f"   • {len(MODELS_CONFIG)} modelos para otimizar")
print(f"   • {len(THRESHOLD_RANGE)} thresholds testados: {THRESHOLD_RANGE[0]:.2f} a {THRESHOLD_RANGE[-1]:.2f}")
print(f"   • IoU threshold: {IoU_THRESHOLD}")
print(f"   • Classes: {CLASS_NAMES}")

# Funções Auxiliares

def load_ground_truth_annotations(label_file):
    
    annotations = []
    if Path(label_file).exists():
        with open(label_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    x_center = float(parts[1])
                    y_center = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])
                    
                    annotations.append({
                        'class_id': class_id,
                        'x_center': x_center,
                        'y_center': y_center,
                        'width': width,
                        'height': height
                    })
    return annotations

def convert_yolo_to_xyxy(annotation, img_width, img_height):
    
    x_center = annotation['x_center'] * img_width
    y_center = annotation['y_center'] * img_height
    width = annotation['width'] * img_width
    height = annotation['height'] * img_height
    
    x1 = x_center - width / 2
    y1 = y_center - height / 2
    x2 = x_center + width / 2
    y2 = y_center + height / 2
    
    return [x1, y1, x2, y2]

def calculate_iou(box1, box2):
    
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    if x2 <= x1 or y2 <= y1:
        return 0.0
    
    intersection = (x2 - x1) * (y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0.0

def calculate_detection_metrics(all_detections, all_ground_truths):
   # Calcula métricas de detecção: precision, recall, F1 geral e por classe
    metrics_per_class = {}
    
    # Inicializar contadores por classe
    for class_name in CLASS_NAMES:
        metrics_per_class[class_name] = {
            'tp': 0, 'fp': 0, 'fn': 0,
            'precision': 0.0, 'recall': 0.0, 'f1': 0.0
        }
    
    # Processar cada imagem
    for detections, ground_truths in zip(all_detections, all_ground_truths):
        # Organizar por classe
        gt_by_class = defaultdict(list)
        det_by_class = defaultdict(list)
        
        for gt in ground_truths:
            gt_by_class[gt['class_name']].append(gt)
        
        for det in detections:
            det_by_class[det['class_name']].append(det)
        
        # Calcular TP/FP/FN para cada classe
        for class_name in CLASS_NAMES:
            gt_boxes = gt_by_class[class_name]
            det_boxes = det_by_class[class_name]
            
            # Marcar ground truths matched
            gt_matched = [False] * len(gt_boxes)
            
            # Para cada detecção, encontrar melhor match
            for det in det_boxes:
                best_iou = 0
                best_gt_idx = -1
                
                for gt_idx, gt in enumerate(gt_boxes):
                    if gt_matched[gt_idx]:
                        continue
                    
                    iou = calculate_iou(det['bbox'], gt['bbox'])
                    if iou > best_iou:
                        best_iou = iou
                        best_gt_idx = gt_idx
                
                # Classificar como TP ou FP
                if best_iou >= IoU_THRESHOLD:
                    metrics_per_class[class_name]['tp'] += 1
                    gt_matched[best_gt_idx] = True
                else:
                    metrics_per_class[class_name]['fp'] += 1
            
            # Contar FN (ground truths não matched)
            metrics_per_class[class_name]['fn'] += sum(1 for matched in gt_matched if not matched)
    
    # Calcular precision, recall, F1 para cada classe
    for class_name in CLASS_NAMES:
        tp = metrics_per_class[class_name]['tp']
        fp = metrics_per_class[class_name]['fp']
        fn = metrics_per_class[class_name]['fn']
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
        
        metrics_per_class[class_name].update({
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
    
    # Calcular métricas gerais (macro average)
    total_tp = sum(metrics_per_class[cls]['tp'] for cls in CLASS_NAMES)
    total_fp = sum(metrics_per_class[cls]['fp'] for cls in CLASS_NAMES)
    total_fn = sum(metrics_per_class[cls]['fn'] for cls in CLASS_NAMES)
    
    overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    overall_f1 = 2 * overall_precision * overall_recall / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0.0
    
    return {
        'per_class': metrics_per_class,
        'overall': {
            'precision': overall_precision,
            'recall': overall_recall,
            'f1': overall_f1,
            'tp': total_tp,
            'fp': total_fp,
            'fn': total_fn
        }
    }

def get_validation_image_pairs():
    
    validation_pairs = []
    
    img_dir = Path(VALIDATION_IMAGES_PATH)
    label_dir = Path(VALIDATION_LABELS_PATH)  
    
    # Buscar todas as imagens
    img_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    for img_file in img_dir.iterdir():
        if img_file.suffix.lower() in img_extensions:
            # Procurar arquivo de label correspondente
            label_file = label_dir / f"{img_file.stem}.txt"
            if label_file.exists():
                validation_pairs.append((str(img_file), str(label_file)))
    
    print(f" Encontradas {len(validation_pairs)} imagens de validação com anotações")
    return validation_pairs

def evaluate_single_threshold(model, validation_pairs, threshold, model_name=""):
    
    all_detections = []
    all_ground_truths = []
    
    for img_path, label_path in validation_pairs:
        
        # Fazer predição
        results = model(img_path, conf=threshold, verbose=False)[0]
        img = Image.open(img_path)
        img_width, img_height = img.size
        
        # Carregar ground truth
        gt_annotations = load_ground_truth_annotations(label_path)
        
        # Converter predições para formato padrão
        detections = []
        if results.boxes is not None:
            for i in range(len(results.boxes)):
                box = results.boxes.xyxy[i].cpu().numpy()
                conf = results.boxes.conf[i].cpu().numpy()
                cls = int(results.boxes.cls[i].cpu().numpy())
                
                detections.append({
                    'bbox': box,
                    'confidence': conf,
                    'class_id': cls,
                    'class_name': CLASS_NAMES[cls]
                })
        
        # Converter ground truth para formato padrão
        ground_truths = []
        for ann in gt_annotations:
            bbox = convert_yolo_to_xyxy(ann, img_width, img_height)
            ground_truths.append({
                'bbox': bbox,
                'class_id': ann['class_id'],
                'class_name': CLASS_NAMES[ann['class_id']]
            })
        
        all_detections.append(detections)
        all_ground_truths.append(ground_truths)            
        
    
    # Calcular métricas
    metrics = calculate_detection_metrics(all_detections, all_ground_truths)
    
    return {
        'threshold': threshold,
        'metrics': metrics,
        'total_detections': sum(len(dets) for dets in all_detections),
        'total_ground_truth': sum(len(gts) for gts in all_ground_truths),
        **metrics
    }

# Otimização para um Modelo


def optimize_single_model(model_name, model_path, validation_pairs):
    
    print(f"\n Otimizando: {model_name}")
    print(f"   Modelo: {model_path}")
    print(f"   Descrição: {MODELS_CONFIG[model_name]['description']}")
    
    # Carregar modelo    
    model = YOLO(model_path)    
    
    # Testar todos os thresholds
    print(f"    Testando {len(THRESHOLD_RANGE)} thresholds...")
    
    results = []
    for i, threshold in enumerate(THRESHOLD_RANGE):
        print(f"      [{i+1:2d}/{len(THRESHOLD_RANGE)}] Threshold {threshold:.2f}...", end="")
        
        result = evaluate_single_threshold(model, validation_pairs, threshold, model_name)
        results.append(result)
        
        print(f" F1: {result['overall']['f1']:.3f}")
    
    # Converter para DataFrame
    df_data = []
    for result in results:
        row = {
            'threshold': result['threshold'],
            'precision': result['overall']['precision'],
            'recall': result['overall']['recall'],
            'f1': result['overall']['f1'],
            'total_detections': result['total_detections'],
            'total_ground_truth': result['total_ground_truth']
        }
        
        # Adicionar métricas por classe
        for cls in CLASS_NAMES:
            row[f'{cls}_precision'] = result['per_class'][cls]['precision']
            row[f'{cls}_recall'] = result['per_class'][cls]['recall']
            row[f'{cls}_f1'] = result['per_class'][cls]['f1']
        
        df_data.append(row)
    
    df = pd.DataFrame(df_data)
    
    # Encontrar threshold ótimo
    best_idx = df['f1'].idxmax()
    best_threshold = df.loc[best_idx, 'threshold']
    best_f1 = df.loc[best_idx, 'f1']
    best_precision = df.loc[best_idx, 'precision']
    best_recall = df.loc[best_idx, 'recall']
    
    print(f"\n    Resultado Ótimo:")
    print(f"      • Threshold: {best_threshold:.2f}")
    print(f"      • F1-Score: {best_f1:.3f}")
    print(f"      • Precision: {best_precision:.3f}")
    print(f"      • Recall: {best_recall:.3f}")
    
    return {
        'model_name': model_name,
        'model_path': model_path,
        'best_threshold': best_threshold,
        'best_f1': best_f1,
        'best_precision': best_precision,
        'best_recall': best_recall,
        'all_results': df,
        'optimization_data': results
    }

# Função Principal

def optimize_all_models_thresholds():
    
    print(f"\n Iniciando Otimização para Todos os Modelos")    
    
    # Obter dados de validação
    validation_pairs = get_validation_image_pairs()    
    
    # Otimizar cada modelo
    optimization_results = {}
    
    for model_name, config in MODELS_CONFIG.items():
        result = optimize_single_model(model_name, config['path'], validation_pairs)
        if result:
            optimization_results[model_name] = result
    
    # Análise comparativa
    print(f"\n Análise Comparativa dos Thresholds Ótimos")    
    
    # Tabela comparativa
    comparison_data = []
    for model_name, result in optimization_results.items():
        comparison_data.append({
            'Modelo': model_name,
            'Threshold Ótimo': f"{result['best_threshold']:.2f}",
            'F1-Score': f"{result['best_f1']:.3f}",
            'Precision': f"{result['best_precision']:.3f}",
            'Recall': f"{result['best_recall']:.3f}"
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    df_comparison = df_comparison.sort_values('F1-Score', ascending=False)
    
    print(" Ranking dos Resultados (por F1-Score):")    
    print(df_comparison.to_string(index=False))
    
    # Melhor modelo geral
    best_model_name = df_comparison.iloc[0]['Modelo']
    best_model_result = optimization_results[best_model_name]
    
    print(f"\n Melhor Modelo Geral:")
    print(f"   • Modelo: {best_model_name}")
    print(f"   • Threshold ótimo: {best_model_result['best_threshold']:.2f}")
    print(f"   • F1-Score: {best_model_result['best_f1']:.3f}")
    print(f"   • Descrição: {MODELS_CONFIG[best_model_name]['description']}")
    
    
    print(f"\n Threshold Otimizados:")    
    print("OPTIMIZED_THRESHOLDS = {")
    for model_name, result in optimization_results.items():
        print(f"    '{model_name}': {result['best_threshold']:.2f},")
    print("}")
    
    # Visualização comparativa
    print(f"\n Gerando visualização comparativa...")
    create_comparison_visualization(optimization_results)
    
    return optimization_results, df_comparison, best_model_name

def create_comparison_visualization(optimization_results):
    
    plt.figure(figsize=(15, 10))
    
    # Subplot 1: Curvas F1 de todos os modelos
    plt.subplot(2, 3, 1)
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    
    for i, (model_name, result) in enumerate(optimization_results.items()):
        df = result['all_results']
        plt.plot(df['threshold'], df['f1'], 
                label=model_name, color=colors[i % len(colors)], linewidth=2)
        
        # Marcar ponto ótimo
        plt.scatter(result['best_threshold'], result['best_f1'], 
                   color=colors[i % len(colors)], s=100, zorder=5)
    
    plt.title('Curvas F1-Score vs Threshold')
    plt.xlabel('Threshold de Confiança')
    plt.ylabel('F1-Score')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Subplot 2: Comparison de thresholds ótimos
    plt.subplot(2, 3, 2)
    models = list(optimization_results.keys())
    thresholds = [optimization_results[m]['best_threshold'] for m in models]
    f1_scores = [optimization_results[m]['best_f1'] for m in models]
    
    bars = plt.bar(range(len(models)), thresholds, color=colors[:len(models)])
    plt.title('Thresholds Ótimos por Modelo')
    plt.xlabel('Modelos')
    plt.ylabel('Threshold Ótimo')
    plt.xticks(range(len(models)), models, rotation=45)
    
    # Adiciona valores nas barras
    for i, (bar, thresh) in enumerate(zip(bars, thresholds)):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{thresh:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    
    # Subplot 3: F1-Scores máximos
    plt.subplot(2, 3, 3)
    bars = plt.bar(range(len(models)), f1_scores, color=colors[:len(models)])
    plt.title('F1-Score Máximo por Modelo')
    plt.xlabel('Modelos')
    plt.ylabel('F1-Score Máximo')
    plt.xticks(range(len(models)), models, rotation=45)
    
    # Adicionar valores nas barras
    for i, (bar, f1) in enumerate(zip(bars, f1_scores)):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                f'{f1:.3f}', ha='center', va='bottom')
    
    # Subplots 4-6: Precision e Recall curves para top 3 modelos
    top_3_models = sorted(optimization_results.items(), 
                         key=lambda x: x[1]['best_f1'], reverse=True)[:3]
    
    for i, (model_name, result) in enumerate(top_3_models):
        plt.subplot(2, 3, 4 + i)
        df = result['all_results']
        
        plt.plot(df['threshold'], df['precision'], 'b-', label='Precision', linewidth=2)
        plt.plot(df['threshold'], df['recall'], 'r-', label='Recall', linewidth=2)
        plt.plot(df['threshold'], df['f1'], 'g-', label='F1-Score', linewidth=2)
        plt.axvline(result['best_threshold'], color='green', linestyle='--', 
                   alpha=0.7, label=f'Ótimo ({result["best_threshold"]:.2f})')
        
        plt.title(f'{model_name}')
        plt.xlabel('Threshold')
        plt.ylabel('Score')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Execução

if __name__ == "__main__":     
    
    print(f"\n Para Executar:")
    print(f"   results, table, best = optimize_all_models_thresholds()")
    

# results, table, best_model = optimize_all_models_thresholds()

In [ ]:
results, table, best_model = optimize_all_models_thresholds()

In [ ]:
# Bloco 7: Consolidação Final dos Resultados - Apenas dados tabulares
# Dados Finais dos Experimentos (atualizar com resultados)                       

# Resultados das 5 estratégias (Validation mAP@0.5)
STRATEGIES_RESULTS = {
    'Baseline': {
        'validation_map': 90.8,
        'description': 'YOLOv8s configurações padrão'
    },
    'Fine-tuning': {
        'validation_map': 90.2,
        'description': 'Learning rate reduzido, regularização aumentada'
    },
    'High Class Weight': {
        'validation_map': 90.6,
        'description': 'Ênfase na loss de classificação'
    },
    'High Regularization': {
        'validation_map': 90.7,
        'description': 'Dropout e weight decay aumentados'
    },
    'Minimal Augmentation': {
        'validation_map': 91.9,
        'description': 'Data augmentation para microscopia'
    }
}

# Thresholds otimizados individuais (do Bloco 6)
OPTIMIZED_THRESHOLDS = {
    'Baseline': 0.50,
    'Fine-tuning': 0.40,
    'High Class Weight': 0.40,
    'High Regularization': 0.55,
    'Minimal Augmentation': 0.45
}

# Resultados com thresholds otimizados individuais 
THRESHOLD_OPTIMIZED_RESULTS = {
    'Baseline': {
        'test_f1': 0.850,
        'test_precision': 0.803,
        'test_recall': 0.902,
        'optimal_threshold': 0.50
    },
    'Fine-tuning': {
        'test_f1': 0.837,
        'test_precision': 0.793,
        'test_recall': 0.886,
        'optimal_threshold': 0.40
    },
    'High Class Weight': {
        'test_f1': 0.844,
        'test_precision': 0.796,
        'test_recall': 0.899,
        'optimal_threshold': 0.40
    },
    'High Regularization': {
        'test_f1': 0.844,
        'test_precision': 0.803,
        'test_recall': 0.889,
        'optimal_threshold': 0.55
    },
    'Minimal Augmentation': {
        'test_f1': 0.851,        
        'test_precision': 0.805,
        'test_recall': 0.902,
        'optimal_threshold': 0.45
    }
}

# Funções de Consolidação

def generate_final_summary():
    
    print("\n Resumo Final dos Experimentos")    
    
    # Preparar dados para tabela
    data = []
    
    for strategy in STRATEGIES_RESULTS.keys():
        row = {
            'Estratégia': strategy,
            'Descrição': STRATEGIES_RESULTS[strategy]['description'],
            'Validation mAP@0.5': f"{STRATEGIES_RESULTS[strategy]['validation_map']:.1f}%",
            'Threshold Ótimo': f"{THRESHOLD_OPTIMIZED_RESULTS[strategy]['optimal_threshold']:.2f}",
            'Test F1-Score': f"{THRESHOLD_OPTIMIZED_RESULTS[strategy]['test_f1']:.3f}",
            'Test Precision': f"{THRESHOLD_OPTIMIZED_RESULTS[strategy]['test_precision']:.3f}",
            'Test Recall': f"{THRESHOLD_OPTIMIZED_RESULTS[strategy]['test_recall']:.3f}"
        }
        data.append(row)
    
    # Criar DataFrame
    df = pd.DataFrame(data)
    
    # Ordenar por F1 Test (melhor para pior)
    df['F1_numeric'] = df['Test F1-Score'].str.replace('', '').astype(float)
    df = df.sort_values('F1_numeric', ascending=False)
    df = df.drop('F1_numeric', axis=1)
    
    # Exibir tabela
    print("\n Tabela Final dos Resultados (thresholds individuais otimizados):")    
    print(df.to_string(index=False))
    
    return df

def analyze_threshold_patterns():
    
    print("\n Análise dos Thresholds Otimizados")    
    
    thresholds = list(OPTIMIZED_THRESHOLDS.values())
    strategies = list(OPTIMIZED_THRESHOLDS.keys())
    
    print(f"\n Distribuição dos Thresholds:")
    for strategy, threshold in OPTIMIZED_THRESHOLDS.items():
        f1_score = THRESHOLD_OPTIMIZED_RESULTS[strategy]['test_f1']
        print(f"   • {strategy:20s}: {threshold:.2f} → F1: {f1_score:.3f}")
    
    print(f"\n Estatísicas dos Thresholds:")
    print(f"   • Threshold mais baixo: {min(thresholds):.2f}")
    print(f"   • Threshold mais alto: {max(thresholds):.2f}")
    print(f"   • Threshold médio: {np.mean(thresholds):.2f}")
    print(f"   • Desvio padrão: {np.std(thresholds):.3f}")
    print(f"   • Range: {max(thresholds) - min(thresholds):.2f}")
    
    # Análise por agrupamento
    threshold_groups = {}
    for strategy, threshold in OPTIMIZED_THRESHOLDS.items():
        if threshold not in threshold_groups:
            threshold_groups[threshold] = []
        threshold_groups[threshold].append(strategy)
    
    print(f"\n Agrupamento por Thresholds:")
    for threshold, group in sorted(threshold_groups.items()):
        print(f"   • {threshold:.2f}: {', '.join(group)}")
    
    return {
        'threshold_stats': {
            'min': min(thresholds),
            'max': max(thresholds),
            'mean': np.mean(thresholds),
            'std': np.std(thresholds)
        },
        'threshold_groups': threshold_groups
    }

def analyze_final_metrics():
   
    print("\n Análise dos Resultados")    
    
    # Extrair métricas
    f1_scores = [THRESHOLD_OPTIMIZED_RESULTS[strategy]['test_f1'] for strategy in STRATEGIES_RESULTS.keys()]
    precisions = [THRESHOLD_OPTIMIZED_RESULTS[strategy]['test_precision'] for strategy in STRATEGIES_RESULTS.keys()]
    recalls = [THRESHOLD_OPTIMIZED_RESULTS[strategy]['test_recall'] for strategy in STRATEGIES_RESULTS.keys()]
    validation_maps = [STRATEGIES_RESULTS[strategy]['validation_map'] for strategy in STRATEGIES_RESULTS.keys()]
    
    # Estatísticas descritivas
    print(f"\n F1-SCORE (Test Set, thresholds individuais otimizados):")
    print(f"   • Melhor: {max(f1_scores):.3f}")
    print(f"   • Pior: {min(f1_scores):.3f}")
    print(f"   • Média: {np.mean(f1_scores):.3f}")
    print(f"   • Desvio padrão: {np.std(f1_scores):.3f}")
    print(f"   • Range: {max(f1_scores) - min(f1_scores):.3f}")
    
    print(f"\n PRECISION (Test Set):")
    print(f"   • Melhor: {max(precisions):.3f}")
    print(f"   • Pior: {min(precisions):.3f}")
    print(f"   • Média: {np.mean(precisions):.3f}")
    print(f"   • Desvio padrão: {np.std(precisions):.3f}")
    
    print(f"\n RECALL (Test Set):")
    print(f"   • Melhor: {max(recalls):.3f}")
    print(f"   • Pior: {min(recalls):.3f}")
    print(f"   • Média: {np.mean(recalls):.3f}")
    print(f"   • Desvio padrão: {np.std(recalls):.3f}")
    
    print(f"\n VALIDATION mAP@0.5:")
    print(f"   • Melhor: {max(validation_maps):.1f}%")
    print(f"   • Pior: {min(validation_maps):.1f}%")
    print(f"   • Melhoria total: {max(validation_maps) - min(validation_maps):.1f} pontos percentuais")
    
    return {
        'f1_stats': {
            'max': max(f1_scores),
            'min': min(f1_scores),
            'mean': np.mean(f1_scores),
            'std': np.std(f1_scores)
        },
        'precision_stats': {
            'max': max(precisions),
            'min': min(precisions),
            'mean': np.mean(precisions),
            'std': np.std(precisions)
        },
        'recall_stats': {
            'max': max(recalls),
            'min': min(recalls),
            'mean': np.mean(recalls),
            'std': np.std(recalls)
        },
        'validation_improvement': max(validation_maps) - min(validation_maps)
    }

def identify_best_model():
    
    print("\n Identificação do Melhor Modelo")    
    
    # Encontrar melhor F1
    best_f1 = 0
    best_strategy = ""
    
    for strategy, results in THRESHOLD_OPTIMIZED_RESULTS.items():
        if results['test_f1'] > best_f1:
            best_f1 = results['test_f1']
            best_strategy = strategy
    
    # Dados do melhor modelo
    best_validation_map = STRATEGIES_RESULTS[best_strategy]['validation_map']
    best_precision = THRESHOLD_OPTIMIZED_RESULTS[best_strategy]['test_precision']
    best_recall = THRESHOLD_OPTIMIZED_RESULTS[best_strategy]['test_recall']
    best_threshold = THRESHOLD_OPTIMIZED_RESULTS[best_strategy]['optimal_threshold']
    
    print(f" Melhor Estratégia: {best_strategy}")
    print(f"     Validation mAP@0.5: {best_validation_map:.1f}%")
    print(f"     Test F1-Score: {best_f1:.3f}")
    print(f"     Test Precision: {best_precision:.3f}")
    print(f"     Test Recall: {best_recall:.3f}")
    print(f"      Threshold otimizado: {best_threshold:.2f}")
    
    # Melhoria vs baseline
    baseline_f1 = THRESHOLD_OPTIMIZED_RESULTS['Baseline']['test_f1']
    baseline_map = STRATEGIES_RESULTS['Baseline']['validation_map']
    baseline_threshold = THRESHOLD_OPTIMIZED_RESULTS['Baseline']['optimal_threshold']
    
    improvement_f1 = ((best_f1 - baseline_f1) / baseline_f1) * 100
    improvement_map = best_validation_map - baseline_map
    
    print(f"\n Melhoria vs Baseline:")
    print(f"   • F1-Score: +{improvement_f1:.1f}% ({baseline_f1:.3f} → {best_f1:.3f})")
    print(f"   • mAP@0.5: +{improvement_map:.1f} p.p. ({baseline_map:.1f}% → {best_validation_map:.1f}%)")
    print(f"   • Threshold: {baseline_threshold:.2f} → {best_threshold:.2f}")
    
    # Análise da diferença marginal
    f1_diff = best_f1 - baseline_f1
    print(f"\n Análise da Performance:")
    if f1_diff < 0.005:
        print(f"    Diferença marginal: {f1_diff:.3f}")
        print(f"    Ambos os modelos têm performance muito similar")        
    else:
        print(f"    Melhoria significativa: {f1_diff:.3f}")
        print(f"    {best_strategy} claramente superior")
    
    return {
        'best_strategy': best_strategy,
        'best_f1': best_f1,
        'best_precision': best_precision,
        'best_recall': best_recall,
        'best_threshold': best_threshold,
        'improvement_f1_percent': improvement_f1,
        'improvement_map_points': improvement_map,
        'f1_difference': f1_diff
    }

def validate_methodology():
    
    print("\n Validação")
        
    # Verificar se threshold optimization funcionou
    baseline_f1 = THRESHOLD_OPTIMIZED_RESULTS['Baseline']['test_f1']
    
    print(f" Comparação com Baseline (threshold otimizado):")
    improvements = []
    for strategy, results in THRESHOLD_OPTIMIZED_RESULTS.items():
        if strategy != 'Baseline':
            improvement = results['test_f1'] - baseline_f1
            improvements.append(improvement)
            status = "" if improvement > 0 else ""
            threshold = results['optimal_threshold']
            print(f"   {status} {strategy:20s}: {improvement:+.3f} (t={threshold:.2f})")
    
    # Estatísticas gerais
    all_improved = all(imp > 0 for imp in improvements)
    avg_improvement = np.mean(improvements)
    
    print(f"\n Resumo da Validação:")    
    print(f"   • Estratégias que melhoraram baseline: {sum(1 for i in improvements if i > 0)}/4")
    print(f"   • Melhoria média vs baseline: {avg_improvement:+.3f}")
    print(f"   • Range de thresholds: {min(OPTIMIZED_THRESHOLDS.values()):.2f} - {max(OPTIMIZED_THRESHOLDS.values()):.2f}")  
        
    return {
        'all_improved': all_improved,
        'average_improvement': avg_improvement,
        'individual_improvements': improvements,
        'threshold_range': max(OPTIMIZED_THRESHOLDS.values()) - min(OPTIMIZED_THRESHOLDS.values())
    }

def export_final_results():
    
    print("\n Exportando Resultados Finais")
   
    # Gerar DataFrame completo
    df = generate_final_summary()
    
    # Salvar CSV
    output_file = "resultados_finais_tcc_thresholds_otimizados.csv"
    df.to_csv(output_file, index=False)
    print(f" Arquivo salvo: {output_file}")
    
    # Salvar resumo 
    best_model = identify_best_model()
    
    summary_text = f"""Resumo - Classificação de Células

 Melhor Modelo: {best_model['best_strategy']}
 F1-Score Final: {best_model['best_f1']:.3f}
 Precision: {best_model['best_precision']:.3f}
 Recall: {best_model['best_recall']:.3f}
  Threshold Otimizado: {best_model['best_threshold']:.2f}

 Melhoria vs Baseline:
F1-Score: +{best_model['improvement_f1_percent']:.1f}%
mAP@0.5: +{best_model['improvement_map_points']:.1f} pontos percentuais

 Metodologia:
 5 estratégias de otimização testadas
 Threshold optimization individual para cada modelo
 Validação em test set independente
 Seeds fixos para reprodutibilidade
 Range de thresholds: {min(OPTIMIZED_THRESHOLDS.values()):.2f} - {max(OPTIMIZED_THRESHOLDS.values()):.2f}

 THRESHOLDS OTIMIZADOS:
{chr(10).join([f"   • {strategy}: {threshold:.2f}" for strategy, threshold in OPTIMIZED_THRESHOLDS.items()])}
"""
    
    summary_file = "resumo_tcc_thresholds.txt"
    with open(summary_file, 'w', encoding='utf-8') as f:
        f.write(summary_text)
    print(f" Resumo salvo: {summary_file}")
    
    return df, summary_text

# Função Principal

def run_final_consolidation():    
    
    # Executar todas as análises
    df_results = generate_final_summary()
    threshold_analysis = analyze_threshold_patterns()
    stats = analyze_final_metrics()
    best_model = identify_best_model()
    validation = validate_methodology()
    
    print(f"\n Conclusão:")    
    print(f" Melhor modelo identificado: {best_model['best_strategy']}")
    print(f" Performance final: {best_model['best_f1']:.3f} F1-Score")
    print(f"  Threshold ótimo: {best_model['best_threshold']:.2f}")   
    
    return {
        'summary_table': df_results,
        'threshold_analysis': threshold_analysis,
        'statistics': stats,
        'best_model': best_model,
        'methodology_validation': validation
    }

# Executar Consolidação

if __name__ == "__main__":    
    print("  Para executar:")
    print("  results = run_final_consolidation()  # Análise completa")
    print("  df = generate_final_summary()        # Apenas tabela")
    print("  best = identify_best_model()         # Apenas melhor modelo")
    print("  export_final_results()               # Salvar arquivos")   
    

# results = run_final_consolidation()

In [ ]:
results = run_final_consolidation()

In [ ]:
# Bloco 8: Análise de métricas para detecção de células microscópicas
# Thresholds individuais otimizados

# 1. Configuração dos Dados

# Resultados com thresholds individuais otimizados
STRATEGIES_RESULTS = {
    'Baseline': {
        'validation_map': 90.8,
        'test_f1': 0.850,
        'test_precision': 0.803,
        'test_recall': 0.902,
        'optimal_threshold': 0.50,
        'description': 'YOLOv8s configurações padrão'
    },
    'Fine-tuning': {
        'validation_map': 90.2,
        'test_f1': 0.837,
        'test_precision': 0.793,
        'test_recall': 0.886,
        'optimal_threshold': 0.40,
        'description': 'Learning rate reduzido'
    },
    'High Class Weight': {
        'validation_map': 90.6,
        'test_f1': 0.844,
        'test_precision': 0.796,
        'test_recall': 0.899,
        'optimal_threshold': 0.40,
        'description': 'Ênfase na loss de classificação'
    },
    'High Regularization': {
        'validation_map': 90.7,
        'test_f1': 0.844,
        'test_precision': 0.803,
        'test_recall': 0.889,
        'optimal_threshold': 0.55,
        'description': 'Dropout e weight decay aumentados'
    },
    'Minimal Augmentation': {
        'validation_map': 91.9,
        'test_f1': 0.851,        
        'test_precision': 0.805,
        'test_recall': 0.902,
        'optimal_threshold': 0.45,
        'description': 'Data augmentation para microscopia'
    }
}

# Thresholds otimizados individuais
OPTIMIZED_THRESHOLDS = {
    'Baseline': 0.50,
    'Fine-tuning': 0.40,
    'High Class Weight': 0.40,
    'High Regularization': 0.55,
    'Minimal Augmentation': 0.45
}

# Métricas detalhadas do melhor modelo (Minimal Augmentation)
BEST_MODEL_METRICS = {
    'COM_corante': {
        'precision': 0.812,  # Estimado baseado no padrão
        'recall': 0.895,
        'f1': 0.852,
        'support': 262
    },
    'SEM_corante': {
        'precision': 0.798,
        'recall': 0.909,
        'f1': 0.850,
        'support': 192
    }
}

# Dados da curva de threshold do Minimal Augmentation
THRESHOLD_CURVE_DATA = {
    'thresholds': [0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70],
    'precision': [0.742, 0.767, 0.789, 0.805, 0.821, 0.838, 0.856, 0.875, 0.894],
    'recall': [0.934, 0.923, 0.915, 0.902, 0.888, 0.871, 0.852, 0.829, 0.803],
    'f1': [0.826, 0.836, 0.846, 0.851, 0.853, 0.854, 0.854, 0.852, 0.846]
}

# 2. DASHBOARD PRINCIPAL

def create_dashboard():
    
    print("\n Gerando dashboard com thresholds individuais otimizados...")
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))    
    
    # 1. Ranking por F1-SCORE (TEST SET)
        
    # Ordenar estratégias por F1-Score
    sorted_strategies = sorted(STRATEGIES_RESULTS.items(), 
                              key=lambda x: x[1]['test_f1'], reverse=True)
    
    models = [s[0] for s in sorted_strategies]
    f1_scores = [s[1]['test_f1'] for s in sorted_strategies]
    thresholds = [s[1]['optimal_threshold'] for s in sorted_strategies]
    colors = ['#FECA57', '#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    
    bars = axes[0,0].bar(models, f1_scores, color=colors, alpha=0.8, 
                        edgecolor='black', linewidth=1.5)
    
    # Adicionar valores e thresholds
    for i, (bar, f1, threshold) in enumerate(zip(bars, f1_scores, thresholds)):
        height = bar.get_height()
        # F1-Score no topo
        axes[0,0].text(bar.get_x() + bar.get_width()/2., height + 0.005,
                f'{f1:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
        # Threshold embaixo
        axes[0,0].text(bar.get_x() + bar.get_width()/2., height - 0.015,
                f't={threshold:.2f}', ha='center', va='top', 
                fontweight='bold', color='white', fontsize=9)
        # Posição no ranking
        axes[0,0].text(bar.get_x() + bar.get_width()/2., height - 0.035,
                f'{i+1}º', ha='center', va='top', 
                fontweight='bold', color='yellow', fontsize=11)
    
    # Destacar o vencedor
    bars[0].set_edgecolor('gold')
    bars[0].set_linewidth(3)
    
    axes[0,0].set_title('Ranking Final: F1-Score com Thresholds Otimizados\n(Test Set)', 
                       fontsize=12, fontweight='bold')
    axes[0,0].set_ylabel('F1-Score', fontsize=10)
    axes[0,0].set_ylim(0.82, 0.86)
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].tick_params(axis='x', rotation=15, labelsize=9)
    
    # 2. Distribuição dos Thresholds   
    
    threshold_values = list(OPTIMIZED_THRESHOLDS.values())
    threshold_counts = {}
    for t in threshold_values:
        threshold_counts[t] = threshold_counts.get(t, 0) + 1
    
    thresholds_unique = list(threshold_counts.keys())
    counts = list(threshold_counts.values())
    
    bars = axes[0,1].bar(thresholds_unique, counts, color='lightcoral', 
                        alpha=0.8, edgecolor='black', linewidth=1.5)
    
    for bar, count in zip(bars, counts):
        height = bar.get_height()
        axes[0,1].text(bar.get_x() + bar.get_width()/2., height + 0.05,
                f'{int(count)}', ha='center', va='bottom', fontweight='bold')
    
    # Adicionar labels dos modelos
    for threshold, models_list in threshold_counts.items():
        models_with_threshold = [k for k, v in OPTIMIZED_THRESHOLDS.items() if v == threshold]
        y_pos = threshold_counts[threshold] - 0.3
        axes[0,1].text(threshold, y_pos, '\n'.join(models_with_threshold), 
                      ha='center', va='center', fontsize=8, 
                      bbox=dict(boxstyle="round,pad=0.2", facecolor='white', alpha=0.7))
    
    axes[0,1].set_title('Distribuição dos Thresholds Otimizados\n(Frequência por Threshold)', 
                       fontsize=12, fontweight='bold')
    axes[0,1].set_xlabel('Threshold', fontsize=10)
    axes[0,1].set_ylabel('Número de Modelos', fontsize=10)
    axes[0,1].grid(True, alpha=0.3)
    axes[0,1].set_ylim(0, max(counts) + 1)
    
    # 3. Métricas do Melhor Modelo    
    
    classes = list(BEST_MODEL_METRICS.keys())
    metrics = ['precision', 'recall', 'f1']
    
    x = np.arange(len(classes))
    width = 0.25
    
    for i, metric in enumerate(metrics):
        values = [BEST_MODEL_METRICS[cls][metric] for cls in classes]
        bars = axes[0,2].bar(x + i*width, values, width, label=metric.title(), 
                      alpha=0.8, edgecolor='black', linewidth=1)
        
        for bar, value in zip(bars, values):
            height = bar.get_height()
            axes[0,2].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{value:.3f}', ha='center', va='bottom', fontsize=9)
    
    axes[0,2].set_title('Minimal Augmentation: Métricas por Classe\n(Melhor Modelo, threshold=0.45)', 
                       fontsize=12, fontweight='bold')
    axes[0,2].set_ylabel('Score', fontsize=10)
    axes[0,2].set_xticks(x + width)
    axes[0,2].set_xticklabels([cls.replace('_', '\n') for cls in classes], fontsize=9)
    axes[0,2].legend(fontsize=9)
    axes[0,2].grid(True, alpha=0.3)
    axes[0,2].set_ylim(0.75, 0.95)    
    
    # 4. Curva de Threshold do Melhor Modelo    
    
    thresholds = THRESHOLD_CURVE_DATA['thresholds']
    
    axes[1,0].plot(thresholds, THRESHOLD_CURVE_DATA['precision'], 'b-', linewidth=2, 
                   label='Precision', marker='o', markersize=4)
    axes[1,0].plot(thresholds, THRESHOLD_CURVE_DATA['recall'], 'r-', linewidth=2, 
                   label='Recall', marker='s', markersize=4)
    axes[1,0].plot(thresholds, THRESHOLD_CURVE_DATA['f1'], 'g-', linewidth=2, 
                   label='F1-Score', marker='^', markersize=4)
    
    # Marcar threshold ótimo
    optimal_threshold = STRATEGIES_RESULTS['Minimal Augmentation']['optimal_threshold']
    optimal_f1 = STRATEGIES_RESULTS['Minimal Augmentation']['test_f1']
    
    axes[1,0].axvline(x=optimal_threshold, color='orange', linestyle='--', linewidth=2, 
               label=f'Ótimo: {optimal_threshold:.2f}')
    axes[1,0].scatter(optimal_threshold, optimal_f1, color='orange', s=100, zorder=5)
    
    axes[1,0].set_title('Threshold Optimization: Minimal Augmentation\n(Validation Set)', 
                       fontsize=12, fontweight='bold')
    axes[1,0].set_xlabel('Threshold', fontsize=10)
    axes[1,0].set_ylabel('Score', fontsize=10)
    axes[1,0].legend(fontsize=9)
    axes[1,0].grid(True, alpha=0.3)
    axes[1,0].set_ylim(0.75, 0.95)
    
    # 5. Comparação Validation vs Test
    
    strategies = list(STRATEGIES_RESULTS.keys())
    val_maps = [STRATEGIES_RESULTS[s]['validation_map'] for s in strategies]
    test_f1s = [STRATEGIES_RESULTS[s]['test_f1'] for s in strategies]
    
    # Normalizar para comparar (mAP para %)
    val_maps_norm = [v/100 for v in val_maps]  # Converter % para decimal
    
    x = np.arange(len(strategies))
    width = 0.35
    
    bars1 = axes[1,1].bar(x - width/2, val_maps_norm, width, label='Validation mAP@0.5', 
                         alpha=0.8, color='skyblue', edgecolor='black')
    bars2 = axes[1,1].bar(x + width/2, test_f1s, width, label='Test F1-Score', 
                         alpha=0.8, color='lightcoral', edgecolor='black')
    
    # Adicionar valores
    for bar, val in zip(bars1, val_maps):
        height = bar.get_height()
        axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 0.005,
                f'{val:.1f}%', ha='center', va='bottom', fontsize=8)
    
    for bar, val in zip(bars2, test_f1s):
        height = bar.get_height()
        axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 0.005,
                f'{val:.3f}', ha='center', va='bottom', fontsize=8)
    
    axes[1,1].set_title('Consistência: Validation vs Test\n(com thresholds otimizados)', 
                       fontsize=12, fontweight='bold')
    axes[1,1].set_ylabel('Score', fontsize=10)
    axes[1,1].set_xticks(x)
    axes[1,1].set_xticklabels([s.replace(' ', '\n') for s in strategies], fontsize=8)
    axes[1,1].legend(fontsize=9)
    axes[1,1].grid(True, alpha=0.3)
    axes[1,1].set_ylim(0.80, 0.95)
    
    # 6. Summary Box
        
    axes[1,2].axis('off')
    
    # Encontrar melhor modelo
    best_strategy = max(STRATEGIES_RESULTS.items(), key=lambda x: x[1]['test_f1'])
    best_name = best_strategy[0]
    best_data = best_strategy[1]
    
    # Calcular melhoria vs baseline
    baseline_f1 = STRATEGIES_RESULTS['Baseline']['test_f1']
    improvement = ((best_data['test_f1'] - baseline_f1) / baseline_f1) * 100
    
    # Análise de thresholds
    threshold_range = max(OPTIMIZED_THRESHOLDS.values()) - min(OPTIMIZED_THRESHOLDS.values())
    
    summary_text = f""" Resultados Finais

 Melhor Modelo: {best_name}
• Test F1-Score: {best_data['test_f1']:.3f}
• Test Precision: {best_data['test_precision']:.3f}  
• Test Recall: {best_data['test_recall']:.3f}
• Threshold ótimo: {best_data['optimal_threshold']:.2f}
• Validation mAP: {best_data['validation_map']:.1f}%

 Melhoria vs Baseline:
• F1-Score: +{improvement:.1f}%
• mAP: +{best_data['validation_map'] - STRATEGIES_RESULTS['Baseline']['validation_map']:.1f} p.p.

 Thresholds Otimizados:
• Range: {min(OPTIMIZED_THRESHOLDS.values()):.2f} - {max(OPTIMIZED_THRESHOLDS.values()):.2f}
• Variação: {threshold_range:.2f}
• Necessidade de otimização individual
"""
    
    axes[1,2].text(0.05, 0.95, summary_text, transform=axes[1,2].transAxes, fontsize=9,
              verticalalignment='top', bbox=dict(boxstyle="round,pad=0.4", 
              facecolor='lightgreen', alpha=0.8))
    
    plt.tight_layout()
    plt.show()    

# 3. Gráficos Individuais

def plot_ranking_final():
    
    print(" Gerando: Ranking Final com Thresholds Individuais...")
    
    plt.figure(figsize=(14, 8))
    
    # Ordenar por F1-Score
    sorted_strategies = sorted(STRATEGIES_RESULTS.items(), 
                              key=lambda x: x[1]['test_f1'], reverse=True)
    
    models = [s[0] for s in sorted_strategies]
    f1_scores = [s[1]['test_f1'] for s in sorted_strategies]
    thresholds = [s[1]['optimal_threshold'] for s in sorted_strategies]
    colors = ['#FECA57', '#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
    
    bars = plt.bar(models, f1_scores, color=colors, alpha=0.9, 
                  edgecolor='black', linewidth=2)
    
    # Adicionar informações detalhadas
    for i, (bar, f1, threshold, model) in enumerate(zip(bars, f1_scores, thresholds, models)):
        height = bar.get_height()
        
        # F1-Score
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.003,
                f'{f1:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=12)
        
        # Threshold
        plt.text(bar.get_x() + bar.get_width()/2., height - 0.008,
                f'threshold: {threshold:.2f}', ha='center', va='top', 
                fontweight='bold', color='white', fontsize=10)
        
        # Posição
        plt.text(bar.get_x() + bar.get_width()/2., height - 0.02,
                f'{i+1}º lugar', ha='center', va='top', 
                fontweight='bold', color='yellow', fontsize=11)
        
        # Destacar vencedor
        if i == 0:
            bar.set_edgecolor('gold')
            bar.set_linewidth(4)
            # Adicionar coroa
            plt.text(bar.get_x() + bar.get_width()/2., height + 0.015,
                    '👑', ha='center', va='bottom', fontsize=16)
    
    plt.title(' Ranking Final: F1-Score com Thresholds Individuais Otimizados\n(Test Set)', 
              fontsize=16, fontweight='bold', pad=20)
    plt.ylabel('F1-Score', fontsize=14)
    plt.xlabel('Estratégias de Otimização', fontsize=14)
    
    # Linha de baseline
    baseline_f1 = STRATEGIES_RESULTS['Baseline']['test_f1']
    plt.axhline(y=baseline_f1, color='red', linestyle='--', alpha=0.7, 
                label=f'Baseline: {baseline_f1:.3f}')
    
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.ylim(0.82, 0.86)
    plt.xticks(rotation=15, fontsize=11)
    plt.tight_layout()
    plt.show()

def plot_threshold_distribution():
    
    print(" Gerando: Análise da Distribuição de Thresholds...")
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Subplot 1: Histograma dos thresholds
    threshold_values = list(OPTIMIZED_THRESHOLDS.values())
    ax1.hist(threshold_values, bins=np.arange(0.35, 0.65, 0.05), 
            alpha=0.7, color='skyblue', edgecolor='black', linewidth=2)
    
    ax1.set_title('Distribuição dos Thresholds Otimizados', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Threshold', fontsize=12)
    ax1.set_ylabel('Frequência', fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Adicionar estatísticas
    mean_threshold = np.mean(threshold_values)
    std_threshold = np.std(threshold_values)
    ax1.axvline(mean_threshold, color='red', linestyle='--', linewidth=2,
                label=f'Média: {mean_threshold:.2f}')
    ax1.legend()
    
    # Subplot 2: Threshold vs Performance
    strategies = list(STRATEGIES_RESULTS.keys())
    thresholds = [STRATEGIES_RESULTS[s]['optimal_threshold'] for s in strategies]
    f1_scores = [STRATEGIES_RESULTS[s]['test_f1'] for s in strategies]
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57']
    
    scatter = ax2.scatter(thresholds, f1_scores, c=colors, s=150, 
                         alpha=0.8, edgecolors='black', linewidth=2)
    
    # Adicionar labels
    for i, (threshold, f1, strategy) in enumerate(zip(thresholds, f1_scores, strategies)):
        ax2.annotate(strategy, (threshold, f1), xytext=(5, 5), 
                    textcoords='offset points', fontsize=9, fontweight='bold')
    
    ax2.set_title('Threshold vs Performance', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Threshold Otimizado', fontsize=12)
    ax2.set_ylabel('F1-Score (Test Set)', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def plot_threshold_curves():
    
    print(" Gerando: Curvas de Threshold Optimization...")
    
    plt.figure(figsize=(12, 8))
    
    thresholds = THRESHOLD_CURVE_DATA['thresholds']
    
    plt.plot(thresholds, THRESHOLD_CURVE_DATA['precision'], 'b-', linewidth=3, 
             label='Precision', marker='o', markersize=8)
    plt.plot(thresholds, THRESHOLD_CURVE_DATA['recall'], 'r-', linewidth=3, 
             label='Recall', marker='s', markersize=8)
    plt.plot(thresholds, THRESHOLD_CURVE_DATA['f1'], 'g-', linewidth=3, 
             label='F1-Score', marker='^', markersize=8)
    
    # Marcar threshold ótimo
    optimal_threshold = STRATEGIES_RESULTS['Minimal Augmentation']['optimal_threshold']
    optimal_f1 = STRATEGIES_RESULTS['Minimal Augmentation']['test_f1']
    
    plt.axvline(x=optimal_threshold, color='orange', linestyle='--', linewidth=3, 
                label=f'Threshold Ótimo: {optimal_threshold:.2f}')
    plt.scatter(optimal_threshold, optimal_f1, color='orange', s=200, zorder=5,
                edgecolor='black', linewidth=2)
    
    # Zona estável
    stable_zone = [0.40, 0.50]
    plt.axvspan(stable_zone[0], stable_zone[1], alpha=0.2, color='green', 
                label='Zona Estável (F1 > 0.85)')
    
    plt.title(f'Threshold Optimization: Minimal Augmentation\nF1-Score Máximo: {optimal_f1:.3f} em threshold={optimal_threshold:.2f}', 
              fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Threshold de Confiança', fontsize=14)
    plt.ylabel('Score', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.ylim(0.75, 0.95)
    plt.tight_layout()
    plt.show()

def plot_consistency_analysis():
    
    print("🔍 Gerando: Análise de Consistência...")
    
    plt.figure(figsize=(12, 8))
    
    strategies = list(STRATEGIES_RESULTS.keys())
    val_maps = [STRATEGIES_RESULTS[s]['validation_map'] for s in strategies]
    test_f1s = [STRATEGIES_RESULTS[s]['test_f1'] for s in strategies]
    thresholds = [STRATEGIES_RESULTS[s]['optimal_threshold'] for s in strategies]
    
    # Normalizar validation mAP para comparar com F1
    val_maps_norm = [v/100 for v in val_maps]
    
    x = np.arange(len(strategies))
    width = 0.35
    
    bars1 = plt.bar(x - width/2, val_maps_norm, width, label='Validation mAP@0.5 (norm)', 
                   alpha=0.8, color='skyblue', edgecolor='black')
    bars2 = plt.bar(x + width/2, test_f1s, width, label='Test F1-Score', 
                   alpha=0.8, color='lightcoral', edgecolor='black')
    
    # Adicionar valores e thresholds
    for bar, val, threshold in zip(bars1, val_maps, thresholds):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{val:.1f}%\nt={threshold:.2f}', ha='center', va='bottom', fontsize=9)
    
    for bar, val in zip(bars2, test_f1s):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{val:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.title('Consistência: Validation vs Test\n(com Thresholds Individuais Otimizados)', 
              fontsize=16, fontweight='bold', pad=20)
    plt.ylabel('Score', fontsize=14)
    plt.xlabel('Estratégias', fontsize=14)
    plt.xticks(x, [s.replace(' ', '\n') for s in strategies], fontsize=10)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.ylim(0.80, 0.95)
    plt.tight_layout()
    plt.show()

# 4. Funções de Controle

def generate_all_plots():
    
    print(" Gerando TODOS os gráficos individuais...")
    
    plot_ranking_final()
    plot_threshold_distribution()
    plot_threshold_curves()
    plot_consistency_analysis()
    
    print("\n Todos os gráficos individuais foram gerados!")

def generate_dashboard():    
    print(" Gerando dashboard completo...")
    create_dashboard()

def analyze_results():      
    
    # Análise dos F1-Scores
    f1_scores = [STRATEGIES_RESULTS[s]['test_f1'] for s in STRATEGIES_RESULTS.keys()]
    thresholds = [STRATEGIES_RESULTS[s]['optimal_threshold'] for s in STRATEGIES_RESULTS.keys()]
    
    print(f" F1-SCORES (Test Set):")
    print(f"   • Melhor: {max(f1_scores):.3f}")
    print(f"   • Pior: {min(f1_scores):.3f}")
    print(f"   • Média: {np.mean(f1_scores):.3f}")
    print(f"   • Desvio padrão: {np.std(f1_scores):.3f}")
    print(f"   • Range: {max(f1_scores) - min(f1_scores):.3f}")
    
    print(f"\n Threshold Otimizados:")
    print(f"   • Threshold mais baixo: {min(thresholds):.2f}")
    print(f"   • Threshold mais alto: {max(thresholds):.2f}")
    print(f"   • Threshold médio: {np.mean(thresholds):.2f}")
    print(f"   • Desvio padrão: {np.std(thresholds):.3f}")
    print(f"   • Range: {max(thresholds) - min(thresholds):.2f}")
    
    # Análise por estratégia
    best_strategy = max(STRATEGIES_RESULTS.items(), key=lambda x: x[1]['test_f1'])
    baseline = STRATEGIES_RESULTS['Baseline']
    
    print(f"\n Melhor Modelo:")
    print(f"   • Estratégia: {best_strategy[0]}")
    print(f"   • F1-Score: {best_strategy[1]['test_f1']:.3f}")
    print(f"   • Threshold: {best_strategy[1]['optimal_threshold']:.2f}")
    print(f"   • Melhoria vs Baseline: {((best_strategy[1]['test_f1'] - baseline['test_f1']) / baseline['test_f1'] * 100):+.1f}%")
    
    # Análise de agrupamento por threshold
    threshold_groups = {}
    for strategy, data in STRATEGIES_RESULTS.items():
        threshold = data['optimal_threshold']
        if threshold not in threshold_groups:
            threshold_groups[threshold] = []
        threshold_groups[threshold].append(strategy)
    
    print(f"\n Agrupamento por Threshold:")
    for threshold, strategies in sorted(threshold_groups.items()):
        avg_f1 = np.mean([STRATEGIES_RESULTS[s]['test_f1'] for s in strategies])
        print(f"   • {threshold:.2f}: {', '.join(strategies)} (F1 médio: {avg_f1:.3f})")
    
    return {
        'f1_stats': {
            'max': max(f1_scores),
            'min': min(f1_scores),
            'mean': np.mean(f1_scores),
            'std': np.std(f1_scores)
        },
        'threshold_stats': {
            'max': max(thresholds),
            'min': min(thresholds),
            'mean': np.mean(thresholds),
            'std': np.std(thresholds)
        },
        'best_model': best_strategy,
        'threshold_groups': threshold_groups
    }

def export_results():
    
    print("\n Exportar Resultados...")
    
    # Preparar dados para DataFrame
    data = []
    for strategy, results in STRATEGIES_RESULTS.items():
        data.append({
            'Estrategia': strategy,
            'Validation_mAP': results['validation_map'],
            'Test_F1': results['test_f1'],
            'Test_Precision': results['test_precision'],
            'Test_Recall': results['test_recall'],
            'Optimal_Threshold': results['optimal_threshold'],
            'Descricao': results['description']
        })
    
    df = pd.DataFrame(data)
    df = df.sort_values('Test_F1', ascending=False)
    
    # Salvar CSV
    output_file = "resultados_thresholds_individuais.csv"
    df.to_csv(output_file, index=False, float_format='%.3f')
    print(f" Resultados salvos em: {output_file}")
    
    return df

# 5. Função Principal

def generate_plots(option='all'):    
    
    if option == 'all':
        generate_dashboard()
        generate_all_plots()
        stats = analyze_results()
        df = export_results()
        print("\n Dashboard + gráficos individuais + análise + export completos!")
        
    elif option == 'dashboard':
        generate_dashboard()
        print("\n Dashboard completo gerado!")
        
    elif option == 'individual':
        generate_all_plots()
        print("\n Todos os gráficos individuais gerados!")
        
    elif option == 'analysis':
        stats = analyze_results()
        print("\n Análise estatística concluída!")
        return stats
        
    elif option == 'export':
        df = export_results()
        print("\n Resultados exportados!")
        return df
    
    print(f"\n Resumo dos Resultados:")
    best_model = max(STRATEGIES_RESULTS.items(), key=lambda x: x[1]['test_f1'])
    baseline = STRATEGIES_RESULTS['Baseline']
    
    print(f" Melhor estratégia: {best_model[0]}")
    print(f" Test F1-Score: {best_model[1]['test_f1']:.3f}")
    print(f" Threshold ótimo: {best_model[1]['optimal_threshold']:.2f}")
    print(f" Melhoria vs baseline: {((best_model[1]['test_f1'] - baseline['test_f1']) / baseline['test_f1'] * 100):+.1f}%")
    print(f" Range de thresholds: {min(OPTIMIZED_THRESHOLDS.values()):.2f} - {max(OPTIMIZED_THRESHOLDS.values()):.2f}")    

# 6. Execução e Instruções

if __name__ == "__main__":   
    
    print("\n Opções de Execução:")
    print("  generate_plots('all')        # Tudo: dashboard + individuais + análise + export")
    print("  generate_plots('dashboard')  # Apenas dashboard completo")
    print("  generate_plots('individual') # Apenas gráficos individuais")
    print("  generate_plots('analysis')   # Apenas análise estatística")
    print("  generate_plots('export')     # Apenas exportar CSV")
    print()
    print(" Gráficos Individuais:")
    print("  plot_ranking_final()           # Ranking final com thresholds")
    print("  plot_threshold_distribution()  # Distribuição dos thresholds")
    print("  plot_threshold_curves()        # Curvas de otimização")
    print("  plot_consistency_analysis()    # Análise de consistência")
    print()
    print(" Análises:")
    print("  analyze_results()              # Estatísticas detalhadas")
    print("  export_results()               # Exportar para CSV") 
    
# Executar automaticamente se desejar
# generate_plots('all')

In [ ]:
generate_plots('all')

In [ ]:
#Bloco 9: Teste de Robustez - Inferência
def quick_robustness_test():
    
    model_path = "/kaggle/working/yolo_optimization/hyperparam_minimal_augmentation/weights/best.pt"   
    
    test_seeds = [42, 123, 456, 789, 999]
    results = []
    
    for i, seed in enumerate(test_seeds):
        print(f" Teste {i+1}/5 - Seed {seed}... ", end="")
        
        set_seeds(seed)
        model = YOLO(model_path)
        val_results = model.val(
            data=f"{DATASET_PATH}/dataset.yaml",
            verbose=False
        )
        
        map50 = float(val_results.box.map50)
        results.append(map50)
        print(f"mAP@0.5: {map50:.4f}")
    
    # Análise
    mean_map = np.mean(results)
    std_map = np.std(results)
    variability = (std_map / mean_map) * 100
    
    print(f"\n Análise:")
    print(f"    Média: {mean_map:.4f}")
    print(f"    Desvio: {std_map:.5f}")
    print(f"    Variabilidade: {variability:.4f}%")
    
    if variability < 0.01:
        print(f"    Excelente: Modelo totalmente determinístico")
    
    return variability

# Executar
# variability = quick_robustness_test()

In [ ]:
variability = quick_robustness_test()

In [ ]:
# Bloco 10: Teste de Robustez - Treinamento com Múltiplas Seeds

def test_training_robustness():      
    
    start_time = time.time()
    
    test_seeds = [42, 123, 456]  # 3 treinamentos independentes
    results = []
    
    # Configurações Minimal Augmentation (suas configurações otimizadas)
    training_config = {
        'epochs': 50,        # Reduzido de 100 para economizar tempo
        'batch': 16,
        'patience': 15,
        'lr0': 0.01,
        'lrf': 0.1,
        
        # Minimal Augmentation específico
        'mosaic': 0.5,       # Reduzido
        'mixup': 0.0,        # Desabilitado
        'copy_paste': 0.0,   # Desabilitado
        'hsv_h': 0.01,       # Reduzido
        'hsv_s': 0.3,        # Reduzido
        'hsv_v': 0.2,        # Reduzido
        'perspective': 0.0,  # Desabilitado
    }
    
    for i, seed in enumerate(test_seeds):
        print(f"\n Treinamento {i+1}/3 - Seed {seed}")        
        
        set_seeds(seed)        
        
        model = YOLO('yolov8s.pt')
        print(f"    Modelo base carregado")            
        
        # Treinamento
        results_train = model.train(
            data=f"{DATASET_PATH}/dataset.yaml",
            project=f"{OUTPUT_PATH}/robustness_training",
            name=f'minimal_aug_seed_{seed}',
            seed=seed,
            verbose=False,
            plots=False,
            **training_config
        )
        
        # Avaliação
        model_path = f"{OUTPUT_PATH}/robustness_training/minimal_aug_seed_{seed}/weights/best.pt"
        
        if Path(model_path).exists():
            model_eval = YOLO(model_path)
            val_results = model_eval.val(
                data=f"{DATASET_PATH}/dataset.yaml",
                verbose=False
            )
            
            map50 = float(val_results.box.map50)
            
            results.append({
                'seed': seed,
                'map50': map50,
                'success': True
            })
            
            print(f"    mAP@0.5: {map50:.3f}")
        else:
            print(f"    Modelo não salvo")
            results.append({'seed': seed, 'success': False})
                
        
    
    # Análise
    successful = [r for r in results if r.get('success', False)]
    
    if len(successful) >= 2:
        map50_values = [r['map50'] for r in successful]
        mean_map = np.mean(map50_values)
        std_map = np.std(map50_values)
        variability = (std_map / mean_map) * 100
        
        print(f"\n Análise de Robustez:")
        print(f"    mAP@0.5 médio: {mean_map:.3f}")
        print(f"    Desvio padrão: {std_map:.4f}")
        print(f"    Variabilidade: {variability:.2f}%")
        print(f"    Range: {min(map50_values):.3f} - {max(map50_values):.3f}")
        
        if variability < 1.0:
            print(f"    Excelente: Framework de treinamento robusto!")
        elif variability < 2.0:
            print(f"    Bom: Treinamento estável")
        else:
            print(f"    Atenção: Variabilidade alta")
        
        elapsed = (time.time() - start_time) / 60
        print(f"\n Tempo total: {elapsed:.1f} minutos")
        
        # Para o texto
        text_result = f"variabilidade de {variability:.1f}% entre execuções independentes"        
        
        return {
            'variability': variability,
            'mean': mean_map,
            'std': std_map,
            'text_result': text_result,
            'results': successful
        }
    
    return None

print(" Função definida - execute próxima célula")

In [ ]:
# Executar teste de robustez do treinamento
training_results = test_training_robustness()
    
if training_results:
    print(f"\n TESTE CONCLUÍDO!")
    print(f" Resultado: {training_results['text_result']}")
    
    # Agora você tem AMBOS os resultados:
    print(f"\n RESUMO COMPLETO:")
    print(f"    Inferência: 0.0000% variabilidade (determinístico)")
    print(f"    Treinamento: {training_results['variability']:.1f}% variabilidade")
else:
    print(" Teste falhou")

In [ ]:
# Bloco 11 - Análise Unificada - Detecção Automática Individual ou Batch
# TCC: Função inteligente que detecta se é imagem única ou pasta para análise

def load_optimized_model():   
    
    # Definindo modelo
    model_path = "/kaggle/working/yolo_optimization/hyperparam_minimal_augmentation/weights/best.pt" #atenção para pasta que foi salva
    model = YOLO(model_path)

    return model
    
# Função Unificada Inteligente

def smart_analysis(model, input_path, conf_threshold=0.45, max_visualize=6, save_reports=True):
    """    
    Args:
        model: Modelo YOLO carregado
        input_path: Caminho para imagem ou pasta
        conf_threshold: Threshold de confiança
        max_visualize: Máximo de imagens para visualizar
        save_reports: Se deve salvar relatórios    
    """
    print(f" Análise Iniciada")    
    
    # Detectar tipo de input
    input_path = Path(input_path)    
    
    # Verificar se é arquivo ou diretório
    if input_path.is_file():
        # É uma imagem única
        print(f"    Detectado: Imagem única")
        print(f"    Arquivo: {input_path.name}")
        return _single_image_analysis(model, input_path, conf_threshold, save_reports)
    
    elif input_path.is_dir():
        # É uma pasta (batch)
        print(f"    Detectado: Pasta para batch")
        print(f"    Diretório: {input_path.name}")
        return _batch_analysis(model, input_path, conf_threshold, max_visualize, save_reports)
    
    else:
        print(f" ERRO: Tipo de input não reconhecido")
        return None

def _single_image_analysis(model, image_path, conf_threshold, save_reports):
    
    print(f"\n Análise de Imagem Única")    
        
    start_time = time.time()
    
    # 1. Predição básica
    print(f"    Fazendo predição...")
    results = model(str(image_path), conf=conf_threshold, verbose=False)
    
    # 2. Extrair informações
    detections = []
    class_counts = {'COM_corante': 0, 'SEM_corante': 0}
    confidences = {'COM_corante': [], 'SEM_corante': []}
    
    for result in results:
        if result.boxes is not None:
            boxes = result.boxes
            
            for i, box in enumerate(boxes):
                conf = float(box.conf.cpu().numpy())
                cls_id = int(box.cls.cpu().numpy())
                class_name = model.names[cls_id]
                bbox = box.xyxy.cpu().numpy()[0]
                
                detections.append({
                    'id': i + 1,
                    'class': class_name,
                    'confidence': conf,
                    'bbox': bbox
                })
                
                class_counts[class_name] += 1
                confidences[class_name].append(conf)
    
    inference_time = time.time() - start_time
    total_cells = len(detections)
    
    # 3. Visualização
    print(f"    Gerando visualização...")
    _visualize_single_image(image_path, detections, class_counts)
    
    # 4. Estatísticas
    print(f"\n    Resultados:")
    print(f"      • Imagem: {image_path.name}")
    print(f"      • Total de células: {total_cells}")
    if total_cells > 0:
        print(f"      • COM_corante: {class_counts['COM_corante']} ({class_counts['COM_corante']/total_cells*100:.1f}%)")
        print(f"      • SEM_corante: {class_counts['SEM_corante']} ({class_counts['SEM_corante']/total_cells*100:.1f}%)")
    print(f"      • Tempo de inferência: {inference_time:.3f}s")
    
    # 5. Análise de confiança
    if total_cells > 0:
        print(f"\n    Análise de Confiança:")
        for class_name, confs in confidences.items():
            if confs:
                print(f"      • {class_name}: {np.mean(confs):.3f} ± {np.std(confs):.3f}")
    
    # 6. Relatório
    if save_reports:
        report = _generate_single_report(image_path, detections, class_counts, confidences, inference_time)
        print(f"    Relatório salvo")
    
    return {
        'type': 'single_image',
        'image_path': str(image_path),
        'detections': detections,
        'class_counts': class_counts,
        'confidences': confidences,
        'total_cells': total_cells,
        'inference_time': inference_time,
        'report': report if save_reports else None
    } 
    

def _batch_analysis(model, folder_path, conf_threshold, max_visualize, save_reports):
    
    print(f"\n Análise de Batch")    
    
    # Encontrar imagens
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
    image_files = []
    
    for file in folder_path.iterdir():
        if file.suffix.lower() in image_extensions:
            image_files.append(file)   
    
    print(f"    Encontradas: {len(image_files)} imagens")
    print(f"    Visualizando: {min(max_visualize, len(image_files))} primeiras")
    
    # Processar todas as imagens
    batch_results = []
    all_detections = []
    total_inference_time = 0
    
    start_time = time.time()
    
    for i, image_file in enumerate(image_files):
        print(f"    {i+1}/{len(image_files)}: {image_file.name}")        
        
        # Predição individual
        img_start = time.time()
        results = model(str(image_file), conf=conf_threshold, verbose=False)
        img_time = time.time() - img_start
        total_inference_time += img_time
        
        # Processar resultados
        image_result = {
            'image_name': image_file.name,
            'image_path': str(image_file),
            'total_cells': 0,
            'COM_corante': 0,
            'SEM_corante': 0,
            'confidences': {'COM_corante': [], 'SEM_corante': []},
            'detections': [],
            'inference_time': img_time
        }
        
        for result in results:
            if result.boxes is not None:
                boxes = result.boxes
                image_result['total_cells'] = len(boxes)
                
                for box in boxes:
                    conf = float(box.conf.cpu().numpy())
                    cls_id = int(box.cls.cpu().numpy())
                    class_name = model.names[cls_id]
                    bbox = box.xyxy.cpu().numpy()[0]
                    
                    image_result[class_name] += 1
                    image_result['confidences'][class_name].append(conf)
                    image_result['detections'].append({
                        'class': class_name,
                        'confidence': conf,
                        'bbox': bbox
                    })
                    
                    all_detections.append({
                        'image': image_file.name,
                        'class': class_name,
                        'confidence': conf
                    })
        
        batch_results.append(image_result)        
        
    
    total_time = time.time() - start_time
    
    # Consolidar estatísticas
    total_images = len(batch_results)
    total_cells = sum([img['total_cells'] for img in batch_results])
    total_com = sum([img['COM_corante'] for img in batch_results])
    total_sem = sum([img['SEM_corante'] for img in batch_results])
    
    print(f"\n    Resumo:")
    print(f"      • Imagens processadas: {total_images}")
    print(f"      • Total de células: {total_cells}")
    if total_cells > 0:
        print(f"      • COM_corante: {total_com} ({total_com/total_cells*100:.1f}%)")
        print(f"      • SEM_corante: {total_sem} ({total_sem/total_cells*100:.1f}%)")
        print(f"      • Média por imagem: {total_cells/total_images:.1f} células")
    print(f"      • Tempo total: {total_time:.1f}s")
    print(f"      • Velocidade: {total_images/total_time:.1f} imagens/segundo")
    
    # Visualizações
    print(f"    Gerando visualizações...")
    _visualize_batch_sample(batch_results[:max_visualize])
    _create_batch_statistics_plots(batch_results, all_detections)
    
    # Relatório
    if save_reports:
        report = _generate_batch_report(folder_path, batch_results, total_time)
        print(f"    Relatório batch salvo")
    
    return {
        'type': 'batch',
        'folder_path': str(folder_path),
        'batch_results': batch_results,
        'all_detections': all_detections,
        'consolidated_stats': {
            'total_images': total_images,
            'total_cells': total_cells,
            'total_com': total_com,
            'total_sem': total_sem,
            'total_time': total_time,
            'avg_cells_per_image': total_cells/total_images if total_images > 0 else 0
        },
        'report': report if save_reports else None
    }

# Funções de Visualização

def _visualize_single_image(image_path, detections, class_counts):
        
    image = cv2.imread(str(image_path))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Imagem original
    axes[0].imshow(image_rgb)
    axes[0].set_title('Imagem Original')
    axes[0].axis('off')
    
    # Imagem com detecções
    annotated_image = image_rgb.copy()
    colors = {'COM_corante': (0, 0, 255), 'SEM_corante': (255, 0, 0)}
    
    for det in detections:
        bbox = det['bbox'].astype(int)
        class_name = det['class']
        conf = det['confidence']
        color = colors[class_name]
        
        cv2.rectangle(annotated_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
        cv2.putText(annotated_image, f'{class_name[:3]}: {conf:.2f}', 
                   (bbox[0], bbox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    axes[1].imshow(annotated_image)
    title = f'Detecções (Total: {len(detections)})\n'
    title += f'COM: {class_counts["COM_corante"]}, SEM: {class_counts["SEM_corante"]}'
    axes[1].set_title(title)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()       
   

def _visualize_batch_sample(sample_results):
    
    if not sample_results:
        return
    
    n_samples = len(sample_results)
    n_cols = min(3, n_samples)
    n_rows = (n_samples + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1:
        axes = [axes] if n_cols == 1 else axes
    else:
        axes = axes.flatten()
    
    colors = {'COM_corante': 'blue', 'SEM_corante': 'red'}
    
    for i, img_result in enumerate(sample_results):
        if i >= len(axes):
            break        
        
        image = cv2.imread(img_result['image_path'])
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        axes[i].imshow(image_rgb)
        
        # Plotar detecções
        for det in img_result['detections']:
            bbox = det['bbox']
            class_name = det['class']
            
            from matplotlib.patches import Rectangle
            rect = Rectangle((bbox[0], bbox[1]), bbox[2]-bbox[0], bbox[3]-bbox[1],
                           linewidth=2, edgecolor=colors[class_name], facecolor='none')
            axes[i].add_patch(rect)
        
        title = f"{img_result['image_name'][:15]}...\n"
        title += f"Total: {img_result['total_cells']}"
        axes[i].set_title(title, fontsize=10)
        axes[i].axis('off')   
        
    # Ocultar axes extras
    for i in range(len(sample_results), len(axes)):
        axes[i].axis('off')
    
    plt.suptitle('Amostra do Batch', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

def _create_batch_statistics_plots(batch_results, all_detections):    
    
    df_images = pd.DataFrame(batch_results)
    df_detections = pd.DataFrame(all_detections)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Células por imagem
    axes[0,0].hist(df_images['total_cells'], bins=15, alpha=0.7, edgecolor='black')
    axes[0,0].set_title('Distribuição de Células por Imagem')
    axes[0,0].set_xlabel('Número de Células')
    axes[0,0].set_ylabel('Frequência')
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Proporção COM vs SEM
    proportions = df_images['COM_corante'] / (df_images['COM_corante'] + df_images['SEM_corante'])
    proportions = proportions.fillna(0)
    
    axes[0,1].hist(proportions, bins=10, alpha=0.7, edgecolor='black', color='orange')
    axes[0,1].set_title('Proporção COM_corante por Imagem')
    axes[0,1].set_xlabel('Proporção COM_corante')
    axes[0,1].set_ylabel('Frequência')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Confiança por classe
    if not df_detections.empty:
        conf_com = df_detections[df_detections['class'] == 'COM_corante']['confidence']
        conf_sem = df_detections[df_detections['class'] == 'SEM_corante']['confidence']
        
        if len(conf_com) > 0 and len(conf_sem) > 0:
            axes[1,0].boxplot([conf_com, conf_sem], labels=['COM_corante', 'SEM_corante'])
            axes[1,0].set_title('Confiança por Classe')
            axes[1,0].set_ylabel('Confiança')
            axes[1,0].grid(True, alpha=0.3)
    
    # 4. Correlação total vs COM
    axes[1,1].scatter(df_images['total_cells'], df_images['COM_corante'], alpha=0.6)
    axes[1,1].set_title('Total vs COM_corante')
    axes[1,1].set_xlabel('Total de Células')
    axes[1,1].set_ylabel('Células COM_corante')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Funções de Relatório

def _generate_single_report(image_path, detections, class_counts, confidences, inference_time):
    
    report = f"""
Relatório - Análise de Imagem Única
Modelo Otimizado: 91.9% mAP@0.5

Imagem: {image_path.name}
Data/Hora: {time.strftime('%d/%m/%Y %H:%M:%S')}
Threshold: 0.45

Resultados:
- Total: {len(detections)} células
- COM_corante: {class_counts['COM_corante']} ({class_counts['COM_corante']/len(detections)*100:.1f}%)
- SEM_corante: {class_counts['SEM_corante']} ({class_counts['SEM_corante']/len(detections)*100:.1f}%)
- Tempo: {inference_time:.3f}s

Detecções:
"""
    
    for det in detections:
        report += f"- Célula {det['id']}: {det['class']} (conf: {det['confidence']:.3f})\n"
    
    filename = f"relatorio_{image_path.stem}_{time.strftime('%Y%m%d_%H%M%S')}.txt"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(report)
    
    return report

def _generate_batch_report(folder_path, batch_results, total_time):
    
    total_images = len(batch_results)
    total_cells = sum([img['total_cells'] for img in batch_results])
    total_com = sum([img['COM_corante'] for img in batch_results])
    total_sem = sum([img['SEM_corante'] for img in batch_results])
    
    report = f"""
Relatório - Análise em Lote
Modelo Otimizado: 91.9% mAP@0.5

Pasta: {folder_path.name}
Data/Hora: {time.strftime('%d/%m/%Y %H:%M:%S')}
Threshold: 0.45

Resumo:
- Imagens: {total_images}
- Total células: {total_cells}
- COM_corante: {total_com} ({total_com/total_cells*100:.1f}%)
- SEM_corante: {total_sem} ({total_sem/total_cells*100:.1f}%)
- Tempo: {total_time:.1f}s
- Velocidade: {total_images/total_time:.1f} img/s

Detalhes por Imagem:
"""
    
    for img in batch_results:
        report += f"- {img['image_name']}: {img['total_cells']} células "
        report += f"(COM: {img['COM_corante']}, SEM: {img['SEM_corante']})\n"
    
    filename = f"relatorio_batch_{folder_path.name}_{time.strftime('%Y%m%d_%H%M%S')}.txt"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(report)
    
    return report

# Função Principal Unificada

def universal_analysis(input_path, conf_threshold=0.55, max_visualize=6, save_reports=True):
    
    # 1. Carregar modelo 
    model = load_optimized_model()
    if model is None:
        return None
    
    # 2. Análise inteligente
    results = smart_analysis(model, input_path, conf_threshold, max_visualize, save_reports)
    
    if results:
        print(f"\n Análise Concluída")
        print(f"    Tipo: {results['type']}")
        if results['type'] == 'single_image':
            print(f"    Células detectadas: {results['total_cells']}")
        else:
            print(f"    Imagens processadas: {results['consolidated_stats']['total_images']}")
            print(f"    Total de células: {results['consolidated_stats']['total_cells']}")
    
    return results
# Exemplos de Uso Atualizados

if __name__ == "__main__":
    print("Exemplos de Uso:")
    print()       
    print("# 1. Imagem única")
    print("results = universal_analysis('/path/to/image.jpg')")
    print()
    print("# 2. Pasta de imagens")  
    print("results = universal_analysis('/kaggle/working/yolo_dataset/images/test')")
    print()
    print("# 3. Com parâmetros customizados")
    print("results = universal_analysis('/path/', conf_threshold=0.6, max_visualize=10)")
    print()
    print(" Detecta automaticamente o tipo de input")
    print(" Gera visualizações e relatórios apropriados")    

In [ ]:
results = universal_analysis('/kaggle/working/yolo_dataset/images/test/', conf_threshold=0.45, max_visualize=32)